In [1]:
import pandas as pd 
import numpy as np 

# DATA SOURCE

##  STATE

- Read Data from Source 1 (State, Capital, StateID)

In [2]:
state=pd.read_json('https://mazfreelance.github.io/malaysia-jajahan-api/v1/states.json',dtype=str)
state=state[state['name']!='LUAR NEGARA']
state

,name,name_long,id,code2,code3,capital
0,JOHOR,JOHOR DARUL TAKZIM,1,JH,JOH,JOHOR BAHRU
1,KEDAH,KEDAH DARUL AMAN,2,KD,KED,ALOR SETAR
2,KELANTAN,KELANTAN DARUL NAIM,3,KT,KEL,KOTA BAHRU
3,MELAKA,MELAKA BANDARAYA BERSEJARAH,4,ML,MLK,BANDAR MELAKA
4,NEGERI SEMBILAN,NEGERI SEMBILAN DARUL KHUSUS,5,NS,NSN,SEREMBAN
5,PAHANG,PAHANG DARUL MAKMUR,6,PH,PAH,KUANTAN
6,PULAU PINANG,PULAU PINANG PULAU MUTIARA,7,PN,PEN,GEORGE TOWN
7,PERAK,PERAK DARUL RIDZUAN,8,PR,PRK,IPOH
8,PERLIS,PERLIS INDERA KAYANGAN,9,PL,PER,KANGAR
9,SELANGOR,SELANGOR DARUL EHSAN,10,SG,SEL,SHAH ALAM


##  DISTRICT

- Read Data from Source 2 (StateID, District, DistrictID)

In [3]:
district=pd.read_json('https://mazfreelance.github.io/malaysia-jajahan-api/v1/states/district.json',dtype=str)
district=district[district['state_id']!='nan'].drop(columns='//comment')
district

,id,state_id,name,code
1,900,9,PERLIS,PER
2,201,2,KOTA SETAR,KSE
3,202,2,KUBANG PASU,KPA
4,203,2,PADANG TERAP,PTE
5,204,2,LANGKAWI,LAN
...,...,...,...,...
122,1307,13,KAPIT,KAP
123,1308,13,SAMARAHAN,SAM
124,1309,13,BINTULU,BIN
125,1310,13,MUKAH,MUK


##  SUBDISTRICT

- Read Data from Source 3 (SubDistrict, SubDistrictID, DistrictID)

In [4]:
mukim=pd.read_json('https://mazfreelance.github.io/malaysia-jajahan-api/v1/states/mukim.json',dtype=str)
mukim=mukim[mukim['name']!='-']#.drop(columns='//comment')
mukim

,id,name,district_id
0,090001,ABI,900
1,090002,ARAU,900
2,090003,BERSERI,900
3,090004,CHUPING,900
4,090005,UTAN AJI,900
...,...,...,...
1985,130947,ANAP,1309
1986,130948,KAKUS,1309
1987,130954,JELALONG,1309
1988,130955,BINIA,1309


In [5]:
mukim[mukim['name'].isnull()]

,id,name,district_id


### Transformation

- Left Join (mukim,district) on (DistrictID,ID)

In [6]:
mukim_district=\
pd.merge(
    mukim.drop(columns='id').rename(columns={'name': 'mukim'}),
    district.drop(columns='code').rename(columns={'name': 'district'}),
    how='left',
    left_on='district_id',
    right_on='id'
)

mukim_district

,mukim,district_id,id,state_id,district
0,ABI,900,900,9,PERLIS
1,ARAU,900,900,9,PERLIS
2,BERSERI,900,900,9,PERLIS
3,CHUPING,900,900,9,PERLIS
4,UTAN AJI,900,900,9,PERLIS
...,...,...,...,...,...
1985,ANAP,1309,1309,13,BINTULU
1986,KAKUS,1309,1309,13,BINTULU
1987,JELALONG,1309,1309,13,BINTULU
1988,BINIA,1309,1309,13,BINTULU


In [7]:
mukim_district[mukim_district['mukim'].isnull()]

,mukim,district_id,id,state_id,district


- Left Join (mukim_district, state) on (StateID, ID)

In [8]:
mukim_district_state=pd.merge(mukim_district.drop(columns=['district_id','id']),state,how='left',left_on='state_id',right_on='id').drop(columns=['name_long','capital','id','code2','code3']).rename(columns={'name': 'state'})
mukim_district_state

,mukim,state_id,district,state
0,ABI,9,PERLIS,PERLIS
1,ARAU,9,PERLIS,PERLIS
2,BERSERI,9,PERLIS,PERLIS
3,CHUPING,9,PERLIS,PERLIS
4,UTAN AJI,9,PERLIS,PERLIS
...,...,...,...,...
1985,ANAP,13,BINTULU,SARAWAK
1986,KAKUS,13,BINTULU,SARAWAK
1987,JELALONG,13,BINTULU,SARAWAK
1988,BINIA,13,BINTULU,SARAWAK


In [9]:
mukim_district_state[mukim_district_state['mukim'].isnull()]

,mukim,state_id,district,state


In [14]:
mukim_district_state[['state_id','state']].drop_duplicates().sort_values(['state_id','state'])

,state_id,state
1581,1,JOHOR
1077,10,SELANGOR
570,11,TERENGGANU
1714,12,SABAH
1760,13,SARAWAK
1311,16,WP PUTRAJAYA
26,2,KEDAH
251,3,KELANTAN
1472,4,MELAKA
1312,5,NEGERI SEMBILAN


### Imputation

- Identify Mukim with Null State

In [10]:
mukim_district_state[mukim_district_state['state_id'].isnull()]

,mukim,state_id,district,state
1294,AMPANG,NaN,NaN,NaN
1295,BATU,NaN,NaN,NaN
1296,CHERAS,NaN,NaN,NaN
1297,ULU KELANG,NaN,NaN,NaN
1298,KUALA LUMPUR,NaN,NaN,NaN
1299,PETALING,NaN,NaN,NaN
1300,SETAPAK,NaN,NaN,NaN
1301,BANDAR KUALA LUMPUR,NaN,NaN,NaN
1302,BANDAR PETALING JAYA,NaN,NaN,NaN
1303,BANDAR BANDAR BAHARU SUNGAI BESI,NaN,NaN,NaN


In [11]:
mukim_district_state[mukim_district_state['mukim'].isnull()]

,mukim,state_id,district,state


- Fill in Null with Logical Value

In [ ]:
mukim_district_state_null =\
[
    ['AMPANG','WP KUALA LUMPUR','14'],
    ['BATU','WP KUALA LUMPUR','14'],
    ['CHERAS','WP KUALA LUMPUR','14'],
    ['ULU KELANG','WP KUALA LUMPUR','14'],
    ['KUALA LUMPUR','WP KUALA LUMPUR','14'],
    ['PETALING','WP KUALA LUMPUR','14'],
    ['SETAPAK','WP KUALA LUMPUR','14'],
    ['BANDAR KUALA LUMPUR','WP KUALA LUMPUR','14'],
    ['BANDAR PETALING JAYA','WP KUALA LUMPUR','14'],
    ['BANDAR BANDAR BAHARU SUNGAI BESI','WP KUALA LUMPUR','14'],
    ['PEKAN BATU','WP KUALA LUMPUR','14'],
    ['PEKAN BATU CAVES','WP KUALA LUMPUR','14'],
    ['PEKAN KEPONG','WP KUALA LUMPUR','14'],
    ['PEKAN KUALA PAUH','WP KUALA LUMPUR','14'],
    ['PEKAN PETALING','WP KUALA LUMPUR','14'],
    ['PEKAN SALAK SOUTH','WP KUALA LUMPUR','14'],
    ['PEKAN SUNGAI PENCHALA','WP KUALA LUMPUR','14'],
    ['LABUAN','WP LABUAN','15'],
    ['DONGGONGON','SABAH','12']
 ]

for mukim in mukim_district_state_null:
    mukim_district_state['state']=np.where(mukim_district_state['mukim']==mukim[0],mukim[1],mukim_district_state['state'])

for mukim in mukim_district_state_null:
    mukim_district_state['state_id']=np.where(mukim_district_state['mukim']==mukim[0],mukim[2],mukim_district_state['state_id'])


In [21]:
mukim_district_state[mukim_district_state['mukim'].isnull()]

,mukim,state_id,district,state


In [22]:
mukim_district_state.drop_duplicates(subset=['mukim','state_id','district','state'], keep='first', inplace=True)
mukim_district_state

,mukim,state_id,district,state
0,ABI,9,PERLIS,PERLIS
1,ARAU,9,PERLIS,PERLIS
2,BERSERI,9,PERLIS,PERLIS
3,CHUPING,9,PERLIS,PERLIS
4,UTAN AJI,9,PERLIS,PERLIS
...,...,...,...,...
1985,ANAP,13,BINTULU,SARAWAK
1986,KAKUS,13,BINTULU,SARAWAK
1987,JELALONG,13,BINTULU,SARAWAK
1988,BINIA,13,BINTULU,SARAWAK


In [ ]:
#mukim_district_state['address_partial']=mukim_district_state['mukim']+','+mukim_district_state['district']+','+mukim_district_state['state']
#mukim_district_state

In [30]:
set(mukim_district_state['state']+','+mukim_district_state['state_id'].to_list())

{'JOHOR,1',
 'KEDAH,2',
 'KELANTAN,3',
 'MELAKA,4',
 'NEGERI SEMBILAN,5',
 'PAHANG,6',
 'PERAK,8',
 'PERLIS,9',
 'PULAU PINANG,7',
 'SABAH,12',
 'SARAWAK,13',
 'SELANGOR,10',
 'TERENGGANU,11',
 'WP KUALA LUMPUR,14',
 'WP LABUAN,15',
 'WP PUTRAJAYA,16'}

## LOCALITY

- Read Data from Source 4 (Location, PostCode, District, State)

In [31]:
location=pd.read_csv('../data_source/Malaysia_Postcode-postcodes.csv',dtype=str,header=None)
location.columns=['postcode','location','district','state']
location

,postcode,location,district,state
0,01000,Alor Redeh,Kangar,PLS
1,01000,Alor Tok Terang,Kangar,PLS
2,01000,Bohor Gelam,Kangar,PLS
3,01000,Jalan Abi Tok Hashim,Kangar,PLS
4,01000,Jalan Alor Sena,Kangar,PLS
...,...,...,...,...
56229,98850,Sungai Seloating,Lawas,SRW
56230,98850,Trusan,Lawas,SRW
56231,98850,Tuaran,Lawas,SRW
56232,98857,Peti Surat,Lawas,SRW


- Check if postcode in location data is missing

In [32]:
location.isnull().sum()

postcode    0
location    0
district    0
state       0
dtype: int64

In [33]:
location[['postcode']].drop_duplicates()

,postcode
0,01000
308,01007
309,01009
310,01500
311,01502
...,...
56198,98750
56199,98800
56203,98850
56232,98857


### Transformation

- DataFrame (location) Transformation to Standardize State as of (mukim_district_state)

In [38]:
location_state =\
[
    ['JHR','JOHOR'],
    ['KDH','KEDAH'],
    ['KTN','KELANTAN'],
    ['KUL','WP KUALA LUMPUR'],
    ['LBN','WP LABUAN'],
    ['MLK','MELAKA'],
    ['NSN','NEGERI SEMBILAN'],
    ['PHG','PAHANG'],
    ['PJY','WP PUTRAJAYA'],
    ['PLS','PERLIS'],
    ['PNG','PULAU PINANG'],
    ['PRK','PERAK'],
    ['SBH','SABAH'],
    ['SGR','SELANGOR'],
    ['SRW','SARAWAK'],
    ['TRG','TERENGGANU']
]

for state in location_state:
    location['state']=np.where(location['state']==state[0],state[1],location['state'])

In [41]:
location['district']=location['district'].str.upper()
location['location']=location['location'].str.upper()
location

,postcode,location,district,state
0,01000,ALOR REDEH,KANGAR,PERLIS
1,01000,ALOR TOK TERANG,KANGAR,PERLIS
2,01000,BOHOR GELAM,KANGAR,PERLIS
3,01000,JALAN ABI TOK HASHIM,KANGAR,PERLIS
4,01000,JALAN ALOR SENA,KANGAR,PERLIS
...,...,...,...,...
56229,98850,SUNGAI SELOATING,LAWAS,SARAWAK
56230,98850,TRUSAN,LAWAS,SARAWAK
56231,98850,TUARAN,LAWAS,SARAWAK
56232,98857,PETI SURAT,LAWAS,SARAWAK


In [42]:
mukim_district_state['state']=mukim_district_state['state'].str.upper()
mukim_district_state['district']=mukim_district_state['district'].str.upper()
mukim_district_state['mukim']=mukim_district_state['mukim'].str.upper()
mukim_district_state

,mukim,state_id,district,state
0,ABI,9,PERLIS,PERLIS
1,ARAU,9,PERLIS,PERLIS
2,BERSERI,9,PERLIS,PERLIS
3,CHUPING,9,PERLIS,PERLIS
4,UTAN AJI,9,PERLIS,PERLIS
...,...,...,...,...
1985,ANAP,13,BINTULU,SARAWAK
1986,KAKUS,13,BINTULU,SARAWAK
1987,JELALONG,13,BINTULU,SARAWAK
1988,BINIA,13,BINTULU,SARAWAK


- Function to Return Postcode Value of Reference Table (location) if String Combination of Main Table (mukim_district_state) Exist 

In [43]:
def find_state_district_location(text, reference_df):
    matches = set()  # Use set directly to avoid duplicates
    for idx, row in reference_df.iterrows():
        if (row['state'].lower() in text.lower()) and (row['district'].lower() in text.lower()) and (row['location'].lower() in text.lower()):
            matches.add((row['postcode']))
    return matches  # Returns set of tuples containing (state, district, location) for that particular postcode

def find_state_location(text, reference_df):
    matches = set()  # Use set directly to avoid duplicates
    for idx, row in reference_df.iterrows():
        if (row['state'].lower() in text.lower()) and (row['location'].lower() in text.lower()):
            matches.add((row['postcode']))
    return matches  # Returns set of tuples containing (state, location) for that particular postcode

def find_district_location(text, reference_df):
    matches = set()  # Use set directly to avoid duplicates
    for idx, row in reference_df.iterrows():
        if (row['district'].lower() in text.lower()) and (row['location'].lower() in text.lower()):
            matches.add((row['postcode']))
    return matches  # Returns set of tuples containing (district, location) for that particular postcode
 

- Test the functions

In [44]:
mukim_district_state_test = mukim_district_state.head(10)
mukim_district_state_test

,mukim,state_id,district,state
0,ABI,9,PERLIS,PERLIS
1,ARAU,9,PERLIS,PERLIS
2,BERSERI,9,PERLIS,PERLIS
3,CHUPING,9,PERLIS,PERLIS
4,UTAN AJI,9,PERLIS,PERLIS
5,JEJAWI,9,PERLIS,PERLIS
6,KAYANG,9,PERLIS,PERLIS
7,KECHOR,9,PERLIS,PERLIS
8,KUALA PERLIS,9,PERLIS,PERLIS
9,KURONG ANAI,9,PERLIS,PERLIS


In [45]:
# test functions workability 

# If mukim_district_state_test is a slice, make a copy first
mukim_district_state_test = mukim_district_state_test.copy()

# Assign new columns directly (no need for .to_list())
mukim_district_state_test['postcode1'] = (
    mukim_district_state_test['mukim'].astype(str) + ',' +
    mukim_district_state_test['district'].astype(str) + ',' +
    mukim_district_state_test['state'].astype(str)
).apply(lambda x: find_state_district_location(x, location))

mukim_district_state_test['postcode2'] = (
    mukim_district_state_test['mukim'].astype(str) + ',' +
    mukim_district_state_test['district'].astype(str) + ',' +
    mukim_district_state_test['state'].astype(str)
).apply(lambda x: find_state_location(x, location))

mukim_district_state_test['postcode3'] = (
    mukim_district_state_test['mukim'].astype(str) + ',' +
    mukim_district_state_test['district'].astype(str) + ',' +
    mukim_district_state_test['state'].astype(str)
).apply(lambda x: find_district_location(x, location))

mukim_district_state_test

,mukim,state_id,district,state,postcode1,postcode2,postcode3
0,ABI,9,PERLIS,PERLIS,{},{},{}
1,ARAU,9,PERLIS,PERLIS,{02600},{02600},{02600}
2,BERSERI,9,PERLIS,PERLIS,{},{},{}
3,CHUPING,9,PERLIS,PERLIS,{},{},{}
4,UTAN AJI,9,PERLIS,PERLIS,{},{},{}
5,JEJAWI,9,PERLIS,PERLIS,{},{01000},{}
6,KAYANG,9,PERLIS,PERLIS,{},{},{06900}
7,KECHOR,9,PERLIS,PERLIS,{},{},{}
8,KUALA PERLIS,9,PERLIS,PERLIS,{02000},{02000},{02000}
9,KURONG ANAI,9,PERLIS,PERLIS,{},{},{}


In [ ]:
# apply functions 

mukim_district_state['postcode1']=np.nan
mukim_district_state['postcode2']=np.nan
mukim_district_state['postcode3']=np.nan

mukim_district_state['postcode1'] = (mukim_district_state['mukim'].astype(str)+','+mukim_district_state['district'].astype(str)+','+mukim_district_state['state'].astype(str)).apply(lambda x: find_state_district_location(x, location)).to_list()
mukim_district_state['postcode2'] = (mukim_district_state['mukim'].astype(str)+','+mukim_district_state['district'].astype(str)+','+mukim_district_state['state'].astype(str)).apply(lambda x: find_state_location(x, location)).to_list()
mukim_district_state['postcode3'] = (mukim_district_state['mukim'].astype(str)+','+mukim_district_state['district'].astype(str)+','+mukim_district_state['state'].astype(str)).apply(lambda x: find_district_location(x, location)).to_list()


- Split Postcode based on Combination of Scenario 1,2 and 3

In [ ]:
mukim_district_state['postcode1'] = mukim_district_state['postcode1'].astype(str).str.replace(r"[\{\}\(\)]|set|'", '', regex=True)
mukim_district_state['postcode2'] = mukim_district_state['postcode2'].astype(str).str.replace(r"[\{\}\(\)]|set|'", '', regex=True)
mukim_district_state['postcode3'] = mukim_district_state['postcode3'].astype(str).str.replace(r"[\{\}\(\)]|set|'", '', regex=True)
mukim_district_state

In [ ]:
#mukim_district_state.to_csv('./data_preprocessed/prep1.csv',index=False)

In [118]:
mukim_district_state=pd.read_csv('../data_preprocessed/prep0.csv',dtype=str)
mukim_district_state.head(10)
#mukim_district_state

,postcode,location,district,state,mukim,postcode1,postcode2,postcode3
0,NaN,ALOR REDEH,KANGAR,PERLIS,NaN,NaN,NaN,NaN
1,NaN,ALOR TOK TERANG,KANGAR,PERLIS,NaN,NaN,NaN,NaN
2,NaN,BOHOR GELAM,KANGAR,PERLIS,NaN,NaN,NaN,NaN
3,NaN,JALAN ABI TOK HASHIM,KANGAR,PERLIS,NaN,NaN,NaN,NaN
4,NaN,JALAN ALOR SENA,KANGAR,PERLIS,NaN,NaN,NaN,NaN
5,NaN,JALAN ARAU,KANGAR,PERLIS,NaN,NaN,NaN,NaN
6,NaN,JALAN BESAR KANGAR,KANGAR,PERLIS,NaN,NaN,NaN,NaN
7,NaN,JALAN BOHOR GUNCAR,KANGAR,PERLIS,NaN,NaN,NaN,NaN
8,NaN,JALAN BUKIT MERANTI,KANGAR,PERLIS,NaN,NaN,NaN,NaN
9,NaN,JALAN HOME GUARD,KANGAR,PERLIS,NaN,NaN,NaN,NaN


In [119]:
mukim_district_state=mukim_district_state[mukim_district_state['mukim'].notnull()]
mukim_district_state.head(10)

,postcode,location,district,state,mukim,postcode1,postcode2,postcode3
56234,NaN,NaN,PERLIS,PERLIS,ABI,NaN,NaN,NaN
56235,02600,NaN,PERLIS,PERLIS,ARAU,02600,02600,02600
56236,NaN,NaN,PERLIS,PERLIS,BERSERI,NaN,NaN,NaN
56237,NaN,NaN,PERLIS,PERLIS,CHUPING,NaN,NaN,NaN
56238,NaN,NaN,PERLIS,PERLIS,UTAN AJI,NaN,NaN,NaN
56239,01000,NaN,PERLIS,PERLIS,JEJAWI,NaN,01000,NaN
56240,06900,NaN,PERLIS,PERLIS,KAYANG,NaN,NaN,06900
56241,NaN,NaN,PERLIS,PERLIS,KECHOR,NaN,NaN,NaN
56242,02000,NaN,PERLIS,PERLIS,KUALA PERLIS,02000,02000,02000
56243,NaN,NaN,PERLIS,PERLIS,KURONG ANAI,NaN,NaN,NaN


In [129]:

mukim_district_state[mukim_district_state['state']=='PERLIS'][['mukim','district','state']]

# update Perlis district and postcode

perlis_mukim_district_postcode = [
    ['ABI', 'KANGAR', '01000', 'PERLIS'],
    ['ARAU', 'ARAU', '02600', 'PERLIS'],
    ['BERSERI', 'KANGAR', '02400', 'PERLIS'],
    ['CHUPING', 'PADANG BESAR', '02500', 'PERLIS'],
    ['UTAN AJI', 'KANGAR', '01000', 'PERLIS'],
    ['JEJAWI', 'ARAU', '01000', 'PERLIS'],
    ['KAYANG', 'KANGAR', '02000', 'PERLIS'],
    ['KECHOR', 'KANGAR', '01000', 'PERLIS'],
    ['KUALA PERLIS', 'KANGAR', '02000', 'PERLIS'],
    ['KURONG ANAI', 'ARAU', '02600', 'PERLIS'],
    ['KURONG BATANG', 'KANGAR', '01000', 'PERLIS'],
    ['NGULANG', 'KANGAR', '02400', 'PERLIS'],
    ['ORAN', 'PADANG BESAR', '02500', 'PERLIS'],
    ['PADANG PAUH', 'KANGAR', '01000', 'PERLIS'],
    ['PADANG SIDING', 'ARAU', '02600', 'PERLIS'],
    ['PAYA', 'KANGAR', '02400', 'PERLIS'],
    ['SANGLANG', 'ARAU', '02600', 'PERLIS'],
    ['SENA', 'KANGAR', '01000', 'PERLIS'],
    ['SERIAB', 'KANGAR', '01000', 'PERLIS'],
    ['SUNGAI ADAM', 'ARAU', '02600', 'PERLIS'],
    ['TITI TINGGI', 'PADANG BESAR', '02100', 'PERLIS'],
    ['WANG BINTONG', 'KANGAR', '01000', 'PERLIS'],
    ['BANDAR ARAU', 'ARAU', '02600', 'PERLIS'],
    ['BANDAR KANGAR', 'KANGAR', '01000', 'PERLIS'],
    ['PEKAN KUALA PERLIS', 'KANGAR', '02000', 'PERLIS'],
    ['PEKAN KAKI BUKIT', 'PADANG BESAR', '02200', 'PERLIS']
]

for mukim in perlis_mukim_district_postcode:
    mukim_district_state['postcode']=np.where((mukim_district_state['mukim']==mukim[0])&(mukim_district_state['state']==mukim[3]),mukim[2],mukim_district_state['postcode'])

for mukim in perlis_mukim_district_postcode:
    mukim_district_state['district']=np.where((mukim_district_state['mukim']==mukim[0])&(mukim_district_state['state']==mukim[3]),mukim[1],mukim_district_state['district'])

- Extract Postcode Value by Priority (1,2,3)

In [130]:
mukim_district_state['postcode']=mukim_district_state['postcode1']
mukim_district_state['postcode']=np.where(mukim_district_state['postcode'].isnull(),mukim_district_state['postcode2'],mukim_district_state['postcode'])
#mukim_district_state['postcode']=np.where(mukim_district_state['postcode'].isnull(),mukim_district_state['postcode3'],mukim_district_state['postcode'])
mukim_district_state.head(10)

,postcode,location,district,state,mukim,postcode1,postcode2,postcode3
56234,NaN,NaN,KANGAR,PERLIS,ABI,NaN,NaN,NaN
56235,02600,NaN,ARAU,PERLIS,ARAU,02600,02600,02600
56236,NaN,NaN,KANGAR,PERLIS,BERSERI,NaN,NaN,NaN
56237,NaN,NaN,PADANG BESAR,PERLIS,CHUPING,NaN,NaN,NaN
56238,NaN,NaN,KANGAR,PERLIS,UTAN AJI,NaN,NaN,NaN
56239,01000,NaN,ARAU,PERLIS,JEJAWI,NaN,01000,NaN
56240,NaN,NaN,KANGAR,PERLIS,KAYANG,NaN,NaN,06900
56241,NaN,NaN,KANGAR,PERLIS,KECHOR,NaN,NaN,NaN
56242,02000,NaN,KANGAR,PERLIS,KUALA PERLIS,02000,02000,02000
56243,NaN,NaN,ARAU,PERLIS,KURONG ANAI,NaN,NaN,NaN


In [131]:
mukim_district_state['postcode']=mukim_district_state['postcode'].str.split(',')
mukim_district_state=mukim_district_state.explode('postcode', ignore_index=True).drop(columns=['postcode1','postcode2','postcode3'])
mukim_district_state.head(10)

,postcode,location,district,state,mukim
0,NaN,NaN,KANGAR,PERLIS,ABI
1,02600,NaN,ARAU,PERLIS,ARAU
2,NaN,NaN,KANGAR,PERLIS,BERSERI
3,NaN,NaN,PADANG BESAR,PERLIS,CHUPING
4,NaN,NaN,KANGAR,PERLIS,UTAN AJI
5,01000,NaN,ARAU,PERLIS,JEJAWI
6,NaN,NaN,KANGAR,PERLIS,KAYANG
7,NaN,NaN,KANGAR,PERLIS,KECHOR
8,02000,NaN,KANGAR,PERLIS,KUALA PERLIS
9,NaN,NaN,ARAU,PERLIS,KURONG ANAI


In [132]:
mukim_district_state.isnull().sum()

postcode     625
location    2160
district      19
state          0
mukim          0
dtype: int64

- DataFrame (mukim_district_state) Transformation to Fill In Value for Null Postcode

In [133]:
# show all rows pandas
pd.set_option('display.max_rows', None)

In [134]:
# perlis

perlis_mukim_district_postcode_null =\
[   
    ['ABI','PERLIS','01000'],['ARAU','PERLIS','02600'],['BANDAR ARAU','PERLIS','02600'],['BANDAR KANGAR','PERLIS','01000'],['BERSERI','PERLIS','02400'],['CHUPING','PERLIS','02500'],['UTAN AJI','PERLIS','01000'],['JEJAWI','PERLIS','01000'],
    ['KAYANG','PERLIS','02000'],['KECHOR','PERLIS','01000'],['KURONG ANAI','PERLIS','02600'],['KURONG BATANG','PERLIS','01000'],['KUALA PERLIS','PERLIS','02000'],['NGULANG','PERLIS','02400'],['ORAN','PERLIS','02500'],['PADANG SIDING','PERLIS','02600'],
    ['PADANG PAUH','PERLIS','01000'],['PAYA','PERLIS','02400'],['PEKAN KUALA PERLIS','PERLIS','02000'],['PEKAN KAKI BUKIT','PERLIS','02200'],['SANGLANG','PERLIS','02800'],['SENA','PERLIS','01000'],['SERIAB','PERLIS','01000'],['SUNGAI ADAM','PERLIS','02600'],
    ['TITI TINGGI','PERLIS','02100'],['WANG BINTONG', 'PERLIS', '01000']
]

for mukim in perlis_mukim_district_postcode_null:
    mukim_district_state['postcode']=np.where((mukim_district_state['mukim']==mukim[0])&(mukim_district_state['state']==mukim[1]),mukim[2],mukim_district_state['postcode'])


mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PERLIS')&(mukim_district_state['mukim'].notnull())]
#mukim_district_state.isnull().sum()

,postcode,location,district,state,mukim


In [135]:
# penang

penang_mukim_district_postcode_null =\
[   
    ['MUKIM 1', 'SEBERANG PERAI TENGAH', '13700'],['MUKIM 2', 'SEBERANG PERAI TENGAH', '13500'],['MUKIM 3', 'SEBERANG PERAI TENGAH', '13500'],['MUKIM 4', 'SEBERANG PERAI TENGAH', '13500'],['MUKIM 5', 'SEBERANG PERAI TENGAH', '14000'],['MUKIM 6', 'SEBERANG PERAI TENGAH', '14000'],
    ['MUKIM 7', 'SEBERANG PERAI TENGAH', '14000'],['MUKIM 8', 'SEBERANG PERAI TENGAH', '14000'],['MUKIM 9', 'SEBERANG PERAI TENGAH', '14000'],['MUKIM 10', 'SEBERANG PERAI TENGAH', '14000'],['MUKIM 11', 'SEBERANG PERAI TENGAH', '14000'],['MUKIM 12', 'SEBERANG PERAI TENGAH', '14100'],
    ['MUKIM 13', 'SEBERANG PERAI TENGAH', '14100'],['MUKIM 14', 'SEBERANG PERAI TENGAH', '14000'],['MUKIM 15', 'SEBERANG PERAI TENGAH', '14000'],['MUKIM 16', 'SEBERANG PERAI TENGAH', '14000'],['MUKIM 17', 'SEBERANG PERAI TENGAH', '14000'],
    ['MUKIM 18', 'SEBERANG PERAI TENGAH', '14000'],['MUKIM 19', 'SEBERANG PERAI TENGAH', '14000'],['MUKIM 20', 'SEBERANG PERAI TENGAH', '14400'],['MUKIM 21', 'SEBERANG PERAI TENGAH', '14100'],['BANDAR PRAI','SEBERANG PERAI TENGAH','13600'],
    ['BANDAR BUKIT MERTAJAM','SEBERANG PERAI TENGAH','14000'],['MUKIM 1', 'SEBERANG PERAI UTARA', '13110'],['MUKIM 2', 'SEBERANG PERAI UTARA', '13200'],['MUKIM 3', 'SEBERANG PERAI UTARA', '13200'],['MUKIM 4', 'SEBERANG PERAI UTARA', '13100'],
    ['MUKIM 5', 'SEBERANG PERAI UTARA', '13200'],['MUKIM 6', 'SEBERANG PERAI UTARA', '13200'],['MUKIM 7', 'SEBERANG PERAI UTARA', '13050'],['MUKIM 8', 'SEBERANG PERAI UTARA', '13200'],['MUKIM 9', 'SEBERANG PERAI UTARA', '13000'],['MUKIM 10', 'SEBERANG PERAI UTARA', '13800'],
    ['MUKIM 11', 'SEBERANG PERAI UTARA', '13800'],['MUKIM 12', 'SEBERANG PERAI UTARA', '13300'],['MUKIM 13', 'SEBERANG PERAI UTARA', '14000'],['MUKIM 14', 'SEBERANG PERAI UTARA', '12300'],['MUKIM 15', 'SEBERANG PERAI UTARA', '12100'],['MUKIM 16', 'SEBERANG PERAI UTARA', '13800'],
    ['BANDAR BUTTERWORTH', 'SEBERANG PERAI UTARA', '12000'],['BANDAR KEPALA BATAS', 'SEBERANG PERAI UTARA', '13200'],['MUKIM 1', 'SEBERANG PERAI SELATAN', '14120'],['MUKIM 2', 'SEBERANG PERAI SELATAN', '14000'],['MUKIM 3', 'SEBERANG PERAI SELATAN', '14200'],
    ['MUKIM 4', 'SEBERANG PERAI SELATAN', '14200'],['MUKIM 5', 'SEBERANG PERAI SELATAN', '14200'],['MUKIM 6', 'SEBERANG PERAI SELATAN', '14200'],['MUKIM 7', 'SEBERANG PERAI SELATAN', '14300'],['MUKIM 8', 'SEBERANG PERAI SELATAN', '14300'],
    ['MUKIM 9', 'SEBERANG PERAI SELATAN', '14300'],['MUKIM 10', 'SEBERANG PERAI SELATAN', '14320'],['MUKIM 11', 'SEBERANG PERAI SELATAN', '14300'],['MUKIM 12', 'SEBERANG PERAI SELATAN', '14200'],['MUKIM 13', 'SEBERANG PERAI SELATAN', '14110'],
    ['MUKIM 14', 'SEBERANG PERAI SELATAN', '14100'],['MUKIM 15', 'SEBERANG PERAI SELATAN', '14120'],['MUKIM 16', 'SEBERANG PERAI SELATAN', '14120'],['BANDAR NIBONG TEBAL', 'SEBERANG PERAI SELATAN', '14300'],['BANDAR SUNGAI BAKAP', 'SEBERANG PERAI SELATAN', '14200'],
    ['MUKIM 13', 'TIMOR LAUT', '11500'],['MUKIM 14', 'TIMOR LAUT', '11060'],['MUKIM 16', 'TIMOR LAUT', '11500'],['MUKIM 17', 'TIMOR LAUT', '11100'],['MUKIM 18', 'TIMOR LAUT', '11200'],['BANDAR BATU FERRINGGI', 'TIMOR LAUT', '11000'],['BANDAR GLUGOR', 'TIMOR LAUT', '10150'],
    ['BANDAR GEORGE TOWN', 'TIMOR LAUT', '10450'],['BANDAR TANJONG BUNGAH', 'TIMOR LAUT', '11200'],['BANDAR AYER ITAM', 'TIMOR LAUT', '10460'],['BANDAR BUKIT BENDERA', 'TIMOR LAUT', '11300'],['BANDAR JELUTONG', 'TIMOR LAUT', '11600'],['BANDAR TANJONG TOKONG', 'TIMOR LAUT', '10470'],
    ['BANDAR TANJONG PINANG', 'TIMOR LAUT', '10470'],['MUKIM 1', 'BARAT DAYA', '11010'],['MUKIM 2', 'BARAT DAYA', '11050'],['MUKIM 3', 'BARAT DAYA', '11010'],['MUKIM 4', 'BARAT DAYA', '11000'],['MUKIM 5', 'BARAT DAYA', '11000'],['MUKIM 6', 'BARAT DAYA', '11000'],
    ['MUKIM 7', 'BARAT DAYA', '11000'],['MUKIM 8', 'BARAT DAYA', '11000'],['MUKIM 9', 'BARAT DAYA', '11000'],['MUKIM 10', 'BARAT DAYA', '11900'],['MUKIM 11', 'BARAT DAYA', '11920'],['MUKIM 12', 'BARAT DAYA', '11900'],['MUKIM A', 'BARAT DAYA', '11010'],
    ['MUKIM B', 'BARAT DAYA', '11000'],['MUKIM C', 'BARAT DAYA', '11000'],['MUKIM D', 'BARAT DAYA', '11000'],['MUKIM E', 'BARAT DAYA', '11000'],['MUKIM F', 'BARAT DAYA', '11000'],['MUKIM G', 'BARAT DAYA', '11000'],['MUKIM H', 'BARAT DAYA', '11000'],
    ['MUKIM I', 'BARAT DAYA', '11000'],['MUKIM J', 'BARAT DAYA', '11000'],['BANDAR BALIK PULAU', 'BARAT DAYA', '11000'],['BANDAR BAYAN LEPAS', 'BARAT DAYA', '11900']
]


for mukim in penang_mukim_district_postcode_null:
    mukim_district_state['postcode']=np.where((mukim_district_state['mukim']==mukim[0])&(mukim_district_state['district']==mukim[1]),mukim[2],mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PULAU PINANG')&(mukim_district_state['mukim'].notnull())]
#mukim_district_state.isnull().sum()

,postcode,location,district,state,mukim


In [136]:
kedah_mukim_district_postcode_null = \
[
    ['AH', 'KEDAH', '06100'],['BINJAL', 'KEDAH', '06000'],['GELONG', 'KEDAH', '06000'],['BUKIT TINGGI', 'KEDAH', '06000'],['HUSBA', 'KEDAH', '06000'],['JERLUN', 'KEDAH', '06150'],['KEPELU', 'KEDAH', '06100'],['KUBANG PASU', 'KEDAH', '06000'],['NAGA', 'KEDAH', '06000'],
    ['PADANG PERAHU', 'KEDAH', '06000'],['PERING', 'KEDAH', '06100'],['PUTAT', 'KEDAH', '06000'],['SANGLANG', 'KEDAH', '06150'],['SUNGAI LAKA', 'KEDAH', '06010'],['TEMIN', 'KEDAH', '06000'],['WANG TEPUS', 'KEDAH', '06000'],['BANDAR CHANGLUN', 'KEDAH', '06010'],
    ['BANDAR BANDAR DARULAMAN', 'KEDAH', '06000'],['JERAM', 'KEDAH', '06000'],['JITRA', 'KEDAH', '06000'],['MALAU', 'KEDAH', '06000'],['PELUBANG', 'KEDAH', '06000'],['TUNJANG', 'KEDAH', '06000'],['BANDAR JITRA', 'KEDAH', '06000'],['BANDAR KODIANG', 'KEDAH', '06100'],
    ['BANDAR TUNJANG', 'KEDAH', '06000'],['BANDAR PADANG SERA', 'KEDAH', '06100'],['BANDAR KEPALA BATAS', 'KEDAH', '06200'],['BANDAR BUKIT KAYU HITAM', 'KEDAH', '06050'],['PEKAN AYER HITAM', 'KEDAH', '06150'],['PEKAN KUALA SANGLANG', 'KEDAH', '06150'],
    ['PEKAN SANGLANG', 'KEDAH', '06150'],['PEKAN KERPAN', 'KEDAH', '06150'],['PEKAN SINTOK', 'KEDAH', '06100'],['PEKAN NAPOH', 'KEDAH', '06000'],['PEKAN SUNGAI KOROK', 'KEDAH', '06150'],['BATANG TUNGGANG KANAN', 'KEDAH', '06300'],['BATANG TUNGGANG KIRI', 'KEDAH', '06300'],
    ['BELIMBING KANAN', 'KEDAH', '06300'],['BELIMBING KIRI', 'KEDAH', '06300'],['KURONG HITAM', 'KEDAH', '06300'],['PADANG TERNAK', 'KEDAH', '06300'],['PADANG TERAP KANAN', 'KEDAH', '06300'],['PADANG TERAP KIRI', 'KEDAH', '06300'],['TEKAI', 'KEDAH', '06400'],
    ['TOLAK', 'KEDAH', '06300'],['PEDU', 'KEDAH', '06300'],['BANDAR KUALA NERANG', 'KEDAH', '06300'],['PEKAN NAKA', 'KEDAH', '06350'],['PEKAN DURIAN BURUNG', 'KEDAH', '06300'],['PEKAN LUBOK MERBAU', 'KEDAH', '06300'],['PEKAN BUKIT TEMBAGA', 'KEDAH', '06300'],
    ['PEKAN PADANG SANAI', 'KEDAH', '06300'],['PEKAN KAMPUNG TANJUNG', 'KEDAH', '06300'],['BUKIT MERIAM', 'KEDAH', '06000'],['KUALA', 'KEDAH', '06000'],['PEKULA', 'KEDAH', '06000'],['SEMELING', 'KEDAH', '06000'],['TELOI KIRI', 'KEDAH', '06000'],['BUJANG', 'KEDAH', '08100'],
    ['GURUN', 'KEDAH', '08300'],['HAJI KUDONG', 'KEDAH', '08500'],['KOTA', 'KEDAH', '08500'],['MERBOK', 'KEDAH', '08400'],['PINANG TUNGGAL', 'KEDAH', '08000'],['RANTAU PANJANG', 'KEDAH', '08500'],['SIDAM KIRI', 'KEDAH', '09400'],['SIMPOR', 'KEDAH', '08500'],
    ['SUNGAI PASIR', 'KEDAH', '08000'],['SUNGAI PETANI', 'KEDAH', '08000'],['BANDAR AMAN JAYA', 'KEDAH', '08000'],['BANDAR BEDONG', 'KEDAH', '08100'],['BANDAR SUNGAI LALANG', 'KEDAH', '08100'],['BANDAR TIKAM BATU', 'KEDAH', '08600'],['PEKAN JENIANG', 'KEDAH', '08700'],
    ['PEKAN KOTA KUALA MUDA', 'KEDAH', '08500'],['PEKAN TANJUNG DAWAI', 'KEDAH', '08200'],['ALOR MALAI', 'KEDAH', '05100'],['ALOR MERAH', 'KEDAH', '05250'],['ANAK BUKIT', 'KEDAH', '06550'],['BUKIT PINANG', 'KEDAH', '06720'],['DERGA', 'KEDAH', '05400'],
    ['GUNONG', 'KEDAH', '06570'],['KOTA SETAR', 'KEDAH', '05675'],['HUTAN KAMPONG', 'KEDAH', '05350'],['KANGKONG', 'KEDAH', '06650'],['KUALA KEDAH', 'KEDAH', '06600'],['KUBANG ROTAN', 'KEDAH', '06250'],['LANGGAR', 'KEDAH', '06500'],['LENGKUAS', 'KEDAH', '05350'],
    ['LEPAI', 'KEDAH', '05350'],['LIMBONG', 'KEDAH', '06500'],['MERGONG', 'KEDAH', '05150'],['PADANG HANG', 'KEDAH', '06250'],['PADANG LALANG', 'KEDAH', '06250'],['PENGKALAN KUNDOR', 'KEDAH', '05400'],['PUMPONG', 'KEDAH', '05250'],['SALA KECHIK', 'KEDAH', '06650'],
    ['SUNGAI BAHARU', 'KEDAH', '06250'],['TAJAR', 'KEDAH', '06500'],['TEBENGAU', 'KEDAH', '06600'],['TELAGA MAS', 'KEDAH', '06400'],['TELOK CHENGAI', 'KEDAH', '06600'],['TITI GAJAH', 'KEDAH', '06550'],['TELOK KECHAI', 'KEDAH', '06600'],['BANDAR ALOR SETAR', 'KEDAH', '05100'],
    ['BANDAR ANAK BUKIT', 'KEDAH', '05100'],['BANDAR KUALA KEDAH', 'KEDAH', '06600'],['BANDAR ALOR MERAH', 'KEDAH', '05250'],['BANDAR BUKIT PINANG', 'KEDAH', '06200'],['BANDAR LANGGAR', 'KEDAH', '06500'],['BANDAR SIMPANG EMPAT', 'KEDAH', '06250'],
    ['PEKAN ALOR JANGGUS', 'KEDAH', '06250'],['PEKAN KOTA SARANG SEMUT', 'KEDAH', '06800'],['PEKAN GUNUNG', 'KEDAH', '05150'],['AYER HANGAT', 'KEDAH', '07000'],['BOHOR', 'KEDAH', '07000'],['KEDAWANG', 'KEDAH', '07000'],['KUAH', 'KEDAH', '07000'],
    ['PADANG MASIRAT', 'KEDAH', '07000'],['ULU MELAKA', 'KEDAH', '07000'],['BANDAR KUAH', 'KEDAH', '07000'],['BANDAR PADANG LALANG', 'KEDAH', '07000'],['PEKAN TELOK DATAI', 'KEDAH', '07000'],['DERANG', 'KEDAH', '06400'],['GAJAH MATI', 'KEDAH', '06400'],
    ['JABI', 'KEDAH', '06400'],['LESONG', 'KEDAH', '06350'],['TUALANG', 'KEDAH', '06400'],['BUKIT LADA', 'KEDAH', '06400'],['BANDAR POKOK SENA', 'KEDAH', '06400'],['PEKAN KEBUN 500', 'KEDAH', '06400'],['DULANG', 'KEDAH', '06900'],['SALA BESAR', 'KEDAH', '06900'],
    ['SINGKIR', 'KEDAH', '06900'],['SUNGAI DAUN', 'KEDAH', '06800'],['YAN', 'KEDAH', '06900'],['BANDAR GUAR CHEMPEDAK', 'KEDAH', '08800'],['BANDAR YAN', 'KEDAH', '06900'],['PEKAN SIMPANG TIGA SUNGAI LIMAU', 'KEDAH', '06800'],['PEKAN SUNGAI LIMAU DALAM', 'KEDAH', '06900'],
    ['PEKAN TEROI', 'KEDAH', '06900'],['PEKAN SINGKIR', 'KEDAH', '06900'],['JENERI', 'KEDAH', '08700'],['SIK', 'KEDAH', '08340'],['SOK', 'KEDAH', '08210'],['TELOI', 'KEDAH', '08340'],['BANDAR SIK', 'KEDAH', '08340'],['PEKAN BATU LAMA SIK', 'KEDAH', '08210'],
    ['PEKAN GULAU', 'KEDAH', '08210'],['PEKAN GAJAH PUTEH', 'KEDAH', '08320'],['PEKAN CHAROK PADANG', 'KEDAH', '08200'],['AYER PUTEH', 'KEDAH', '06700'],['BUKIT RAYA', 'KEDAH', '06700'],['GUAR KEPAYANG', 'KEDAH', '06700'],['PADANG KERBAU', 'KEDAH', '06700'],
    ['PADANG PELIANG', 'KEDAH', '06750'],['PADANG PUSING', 'KEDAH', '06700'],['RAMBAI', 'KEDAH', '06400'],['TOBIAR', 'KEDAH', '06700'],['BANDAR PEDANG', 'KEDAH', '06700'],['PEKAN BUKIT JENUN', 'KEDAH', '06720'],['PEKAN KUBUR PANJANG', 'KEDAH', '06400'],
    ['PEKAN TANAH MERAH', 'KEDAH', '06700'],['PEKAN TOKAI', 'KEDAH', '06650'],['PEKAN KOBAH', 'KEDAH', '06800'],['PEKAN KAMPUNG BARU', 'KEDAH', '06800'],['PEKAN SUNGAI TIANG', 'KEDAH', '06800'],['BAKAI', 'KEDAH', '09120'],['BALING', 'KEDAH', '09100'],
    ['BONGOR', 'KEDAH', '09100'],['KUPANG', 'KEDAH', '09200'],['PULAI', 'KEDAH', '09100'],['SIONG', 'KEDAH', '09100'],['TAWAR', 'KEDAH', '09100'],['TELOI KANAN', 'KEDAH', '09310'],['BANDAR BALING', 'KEDAH', '09100'],['BANDAR KUALA KETIL', 'KEDAH', '09300'],
    ['BANDAR KUPANG', 'KEDAH', '09200'],['PEKAN KAMPUNG BARU KEJAI', 'KEDAH', '09120'],['PEKAN PULAI', 'KEDAH', '09100'],['PEKAN TAWAR', 'KEDAH', '09100'],['PEKAN KUALA PEGANG', 'KEDAH', '09100'],['PEKAN PARIT PANJANG', 'KEDAH', '09200'],['PEKAN KAMPUNG LALANG', 'KEDAH', '09100'],
    ['PEKAN MALAU', 'KEDAH', '09000'],['BAGAN SENA', 'KEDAH', '09000'],['JUNJONG', 'KEDAH', '09000'],['KARANGAN', 'KEDAH', '09410'],['KELADI', 'KEDAH', '09000'],['KULIM', 'KEDAH', '09000'],['LUNAS', 'KEDAH', '09600'],['MAHANG', 'KEDAH', '09700'],['NAGA LILIT', 'KEDAH', '09400'],
    ['PADANG CHINA', 'KEDAH', '09600'],['PADANG MEHA', 'KEDAH', '09010'],['SEDIM', 'KEDAH', '09700'],['SIDAM KANAN', 'KEDAH', '09400'],['SUNGAI SELUANG', 'KEDAH', '09600'],['SUNGAI ULAR', 'KEDAH', '09000'],['TERAP', 'KEDAH', '09800'],['BANDAR KULIM', 'KEDAH', '09000'],
    ['BANDAR LUNAS', 'KEDAH', '09600'],['BANDAR PADANG SERAI', 'KEDAH', '09400'],['PEKAN JUNJONG', 'KEDAH', '09000'],['PEKAN KARANGAN', 'KEDAH', '09410'],['PEKAN LABU BESAR', 'KEDAH', '09010'],['PEKAN MAHANG', 'KEDAH', '09700'],['PEKAN MERBAU PULAS', 'KEDAH', '09300'],
    ['PEKAN SUNGAI KARANGAN', 'KEDAH', '09410'],['PEKAN SUNGAI KOB', 'KEDAH', '09700'],['PEKAN PADANG MEHA', 'KEDAH', '09010'],['BAGAN SAMAK', 'KEDAH', '34950'],['KUALA SELAMA', 'KEDAH', '09800'],['PERMATANG PASIR', 'KEDAH', '34950'],['RELAU', 'KEDAH', '14290'],
    ['SERDANG', 'KEDAH', '09810'],['SUNGAI BATU', 'KEDAH', '09800'],['SUNGAI KECHIL', 'KEDAH', '34950'],['BANDAR BANDAR BAHARU', 'KEDAH', '34950'],['BANDAR SERDANG', 'KEDAH', '09800'],['PEKAN LUBUK BUNTAR', 'KEDAH', '09800'],['PEKAN SELAMA', 'KEDAH', '09800'],
    ['PEKAN SUNGAI KECIL ILIR', 'KEDAH', '14390'],['PEKAN RELAU', 'KEDAH', '14290']
]

for mukim in kedah_mukim_district_postcode_null:
    mukim_district_state['postcode']=np.where((mukim_district_state['mukim']==mukim[0])&(mukim_district_state['state']==mukim[1]),mukim[2],mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='KEDAH')&(mukim_district_state['mukim'].notnull())]
#mukim_district_state.isnull().sum()

,postcode,location,district,state,mukim


In [137]:
perak_mukim_district_postcode_null = \
[
    ['BATANG PADANG', 'PERAK', '35000'],['SLIM', 'PERAK', '35800'],['BANDAR CHENDERIAN', 'PERAK', '35350'],['BANDAR TANJONG MALIM', 'PERAK', '35900'],['CHENDERIANG', 'PERAK', '35300'],['HULU BERNAM BARAT', 'PERAK', '35900'],['HULU BERNAM TIMOR', 'PERAK', '35900'],
    ['SUNGKAI', 'PERAK', '35600'],['BANDAR BIDOR', 'PERAK', '35000'],['BANDAR SUNGKAI', 'PERAK', '35600'],['BANDAR TAPAH', 'PERAK', '35000'],['BANDAR TEMOH', 'PERAK', '35350'],['PEKAN AYER KUNING', 'PERAK', '31900'],['PEKAN BANIR', 'PERAK', '35400'],
    ['PEKAN BANTANG', 'PERAK', '35800'],['PEKAN BIKAM', 'PERAK', '35600'],['PEKAN SLIM', 'PERAK', '35800'],['PEKAN SUNGAI LESONG', 'PERAK', '31900'],['PEKAN TAPAH ROAD', 'PERAK', '35400'],['PEKAN TEMOH STATION', 'PERAK', '35350'],['PEKAN TEROLAK', 'PERAK', '35700'],
    ['BIDOR', 'PERAK', '35000'],['BERUAS', 'PERAK', '32700'],['PENGKALAN BAHARU', 'PERAK', '34900'],['LEKIR', 'PERAK', '32000'],['LUMUT', 'PERAK', '32200'],['SITIAWAN', 'PERAK', '32000'],['BANDAR LUMUT', 'PERAK', '32200'],['PEKAN AYER TAWAR', 'PERAK', '32400'],
    ['PEKAN BERUAS', 'PERAK', '32700'],['PEKAN CHANGKAT KERUING', 'PERAK', '32500'],['PEKAN DAMAR LAUT', 'PERAK', '32200'],['PEKAN KAMPONG BAHARU', 'PERAK', '32200'],['PEKAN KAMPONG KOH', 'PERAK', '32000'],['PEKAN KAMPONG SITIAWAN', 'PERAK', '32000'],
    ['PEKAN PANGKOR', 'PERAK', '32300'],['PEKAN PANTAI REMIS', 'PERAK', '34900'],['PEKAN PASIR BOGAK', 'PERAK', '32300'],['PEKAN GURNEY', 'PERAK', '32000'],['PEKAN PANGKALAN BAHARU', 'PERAK', '34900'],['PEKAN SEGARI', 'PERAK', '32200'],
    ['PEKAN SITIAWAN', 'PERAK', '32000'],['PEKAN SUNGAI PINANG KECHIL', 'PERAK', '32300'],['TANJONG TUALANG', 'PERAK', '31800'],['BANDAR IPOH (U)', 'PERAK', '30020'],['BANDAR IPOH (S)', 'PERAK', '31350'],['BANDAR MENGELEMBU', 'PERAK', '31450'],
    ['BANDAR PAPAN', 'PERAK', '31550'],['BANDAR SEPUTEH', 'PERAK', '31560'],['BANDAR SUNGAI RAYA', 'PERAK', '31300'],['BANDAR TANJONG RAMBUTAN', 'PERAK', '31250'],['BANDAR TERONOH', 'PERAK', '31750'],['HULU KINTA', 'PERAK', '31150'],['SUNGAI RAIA', 'PERAK', '31300'],
    ['SUNGAI TERAP', 'PERAK', '31000'],['BANDAR BATU GAJAH', 'PERAK', '31000'],['BANDAR CHEMOR', 'PERAK', '31200'],['BANDAR JELAPANG', 'PERAK', '30020'],['BANDAR LAHAT', 'PERAK', '31500'],['BANDAR PUSING', 'PERAK', '31550'],['BANDAR TAMBUN', 'PERAK', '31400'],
    ['PEKAN KANTHAN', 'PERAK', '31200'],['PEKAN SIMPANG PULAI', 'PERAK', '31300'],['PEKAN TANJONG TUALANG', 'PERAK', '31800'],['BERIAH', 'PERAK', '34300'],['GUNONG SEMANGGOL', 'PERAK', '34400'],['SELINSING', 'PERAK', '34400'],['TANJONG PIANDANG', 'PERAK', '34350'],
    ['BAGAN SERAI', 'PERAK', '34300'],['BAGAN TIANG', 'PERAK', '34250'],['KUALA KURAU', 'PERAK', '34350'],['PARIT BUNTAR', 'PERAK', '34200'],['BANDAR BAGAN SERAI', 'PERAK', '34300'],['BANDAR KUALA KURAU', 'PERAK', '34350'],['BANDAR PARIT BUNTAR', 'PERAK', '34200'],
    ['PEKAN BUKIT MERAH', 'PERAK', '34400'],['PEKAN JALAN BARU', 'PERAK', '34300'],['PEKAN SIMPANG LIMA', 'PERAK', '34200'],['PEKAN SUNGAI GEDONG', 'PERAK', '34300'],['PEKAN TANJONG PIANDANG', 'PERAK', '34250'],['BAGAN DATUK', 'PERAK', '36100'],
    ['CHANGKAT JONG', 'PERAK', '36700'],['RUNGKUP', 'PERAK', '36200'],['SUNGAI MANIK', 'PERAK', '36750'],['TELOK BARU', 'PERAK', '36300'],['DURIAN SEBATANG', 'PERAK', '36000'],['HUTAN MELINTANG', 'PERAK', '36400'],['LABU KUBONG', 'PERAK', '36600'],
    ['SUNGAI DURIAN', 'PERAK', '36000'],['BANDAR TELUK INTAN', 'PERAK', '36000'],['PEKAN BAGAN DATUK', 'PERAK', '36100'],['PEKAN BATAK RABIT', 'PERAK', '36000'],['PEKAN BATU DUA PULOH', 'PERAK', '36200'],['PEKAN CHIKUS', 'PERAK', '36750'],
    ['PEKAN DEGONG', 'PERAK', '36700'],['PEKAN HUTAN MELINTANG', 'PERAK', '36400'],['PEKAN JENDARATA', 'PERAK', '36400'],['PEKAN KAMPUNG SUNGAI HAJI MOHAMED', 'PERAK', '36200'],['PEKAN LANGKAP', 'PERAK', '36700'],['PEKAN SELEKOH', 'PERAK', '36200'],
    ['PEKAN SIMPANG EMPAT', 'PERAK', '36400'],['PEKAN SIMPANT TIGA (RUNGKUP)', 'PERAK', '36200'],['PEKAN SUNGAI SUMUN', 'PERAK', '36300'],['BELUKAR SEMANG', 'PERAK', '33200'],['BELUM', 'PERAK', '33300'],['DURIAN PIPIT', 'PERAK', '33030'],['GRIK', 'PERAK', '33300'],
    ['KERUNAI', 'PERAK', '33310'],['KENERING', 'PERAK', '33410'],['TEMELONG', 'PERAK', '33400'],['TEMENGOR', 'PERAK', '33300'],['BANDAR LAWN', 'PERAK', '33410'],['PENGKALAN HULU', 'PERAK', '33100'],['LENGGONG', 'PERAK', '33400'],['BANDAR GERIK', 'PERAK', '33300'],
    ['BANDAR KELIAN INTAN', 'PERAK', '33100'],['BANDAR PENGKALAN HULU', 'PERAK', '33100'],['BANDAR LENGGONG', 'PERAK', '33400'],['BANDAR', 'PERAK', '36800'],['BELANJA', 'PERAK', '32800'],['JAYA BARU', 'PERAK', '32000'],['KAMPONG GAJAH', 'PERAK', '36800'],
    ['LAMBOR KIRI', 'PERAK', '32600'],['PASIR PANJANG HULU', 'PERAK', '36800'],['BOTA', 'PERAK', '32600'],['KOTA SETIA', 'PERAK', '36810'],['LAMBOR KANAN', 'PERAK', '32900'],['LAYANG LAYANG', 'PERAK', '32800'],['PASIR SALAK', 'PERAK', '36800'],
    ['PULAU TIGA', 'PERAK', '36800'],['BANDAR BANDAR SERI ISKANDAR', 'PERAK', '32610'],['PEKAN BOTA KANAN', 'PERAK', '32600'],['PEKAN KAMPONG BULOH AKAR', 'PERAK', '32800'],['PEKAN KOTA SETIA', 'PERAK', '36810'],['PEKAN PARIT', 'PERAK', '32900'],
    ['PEKAN TANJONG BELANJA', 'PERAK', '32800'],['CHENGAR GALAH', 'PERAK', '33500'],['KAMPONG BUAYA', 'PERAK', '33700'],['KOTA LAMA KANAN', 'PERAK', '33040'],['KOTA LAMA KIRI', 'PERAK', '33000'],['LUBOK MERBAU', 'PERAK', '33000'],['PULAU KAMIRI', 'PERAK', '31100'],
    ['SAIONG', 'PERAK', '33040'],['SENGGANG', 'PERAK', '33000'],['SUNGAI SIPUT', 'PERAK', '33000'],['BANDAR KUALA KANGSAR', 'PERAK', '33000'],['BANDAR SUNGAI SIPUT', 'PERAK', '31100'],['PEKAN GUNONG PONDOK', 'PERAK', '33700'],['PEKAN JERLUN', 'PERAK', '33000'],
    ['PEKAN KARAI', 'PERAK', '31050'],['PEKAN KATI', 'PERAK', '33020'],['PEKAN LUBOK MERBAU', 'PERAK', '33010'],['PEKAN MANONG', 'PERAK', '33800'],['PEKAN PADANG RENGAS', 'PERAK', '33700'],['PEKAN SALAK', 'PERAK', '31050'],['ASAM KUMBANG', 'PERAK', '34000'],
    ['BATU KURAU', 'PERAK', '34500'],['BUKIT GANTANG', 'PERAK', '34030'],['JEBONG', 'PERAK', '34750'],['KAMUNTING', 'PERAK', '34600'],['PENGKALAN AOR', 'PERAK', '34000'],['SIMPANG', 'PERAK', '34000'],['SUNGAI LIMAU', 'PERAK', '34400'],
    ['SUNGAI TINGGI', 'PERAK', '34800'],['TERONG', 'PERAK', '34800'],['TUPAI', 'PERAK', '34000'],['BANDAR KAMUNTING', 'PERAK', '34600'],['BANDAR KUALA SEPETANG', 'PERAK', '34650'],['BANDAR MATANG', 'PERAK', '34700'],['BANDAR TAIPING', 'PERAK', '34000'],
    ['PEKAN BATU KURAU', 'PERAK', '34500'],['PEKAN CHANGKAT JERING', 'PERAK', '34850'],['PEKAN PONDOK TANJONG', 'PERAK', '34010'],['PEKAN SIMPANG', 'PERAK', '34000'],['PEKAN TERUNG', 'PERAK', '34800'],['HULU IJOK', 'PERAK', '34510'],
    ['HULU SELAMA', 'PERAK', '34140'],['SELAMA', 'PERAK', '34100'],['BANDAR SELAMA', 'PERAK', '34100'],['PEKAN RANTAU PANJANG', 'PERAK', '34140'],['PEKAN SUNGAI BAYUR', 'PERAK', '34140'],['KAMPAR', 'PERAK', '31900'],['TEJA', 'PERAK', '31600'],
    ['BANDAR GOPENG', 'PERAK', '31600'],['BANDAR KAMPAR', 'PERAK', '31900'],['PEKAN KOTA BAHARU', 'PERAK', '31600'],['PEKAN MALIM NAWAR', 'PERAK', '31700']
]


for mukim in perak_mukim_district_postcode_null:
    mukim_district_state['postcode']=np.where((mukim_district_state['mukim']==mukim[0])&(mukim_district_state['state']==mukim[1]),mukim[2],mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PERAK')&(mukim_district_state['mukim'].notnull())]	
#mukim_district_state.isnull().sum()

,postcode,location,district,state,mukim


In [138]:
selangor_mukim_district_postcode_null = \
[
    ['KLANG', 'SELANGOR', '41400'],['BANDAR KLANG', 'SELANGOR', '41000'],['BANDAR PORT SWETTENHAM', 'SELANGOR', '42000'],['BANDAR SHAH ALAM', 'SELANGOR', '40000'],['KAPAR', 'SELANGOR', '42200'],['BANDAR SULTAN SULAIMAN', 'SELANGOR', '42000'],
    ['PEKAN BUKIT KEMUNING', 'SELANGOR', '40460'],['PEKAN KAPAR', 'SELANGOR', '42200'],['PEKAN MERU', 'SELANGOR', '41050'],['PEKAN TELOK MENEGUN', 'SELANGOR', '40460'],['PEKAN BATU EMPAT', 'SELANGOR', '42100'],['PEKAN PANDAMARAN', 'SELANGOR', '42000'],
    ['BANDAR', 'SELANGOR', '42700'],['JUGRA', 'SELANGOR', '42960'],['TANJONG DUABELAS', 'SELANGOR', '42700'],['BANDAR TANJONG SEPAT', 'SELANGOR', '42800'],['KELANANG', 'SELANGOR', '42700'],['MORIB', 'SELANGOR', '42700'],
    ['TELOK PANGLIMA GARANG', 'SELANGOR', '42500'],['BANDAR BANTING', 'SELANGOR', '42700'],['BANDAR JENJAROM', 'SELANGOR', '42600'],['BANDAR SIJANGKANG', 'SELANGOR', '42500'],['BANDAR TELOK PANGLIMA GARANG', 'SELANGOR', '42500'],
    ['PEKAN BUKIT CHANGGANG', 'SELANGOR', '42700'],['PEKAN CHODOI', 'SELANGOR', '42700'],['PEKAN JENJAROM', 'SELANGOR', '42600'],['PEKAN KANCHONG', 'SELANGOR', '42700'],['PEKAN KANCHONG DARAT', 'SELANGOR', '42700'],
    ['PEKAN KELANANG BATU ENAM', 'SELANGOR', '42700'],['PEKAN MORIB', 'SELANGOR', '42700'],['PEKAN SIJANGKANG', 'SELANGOR', '42500'],['PEKAN PERMATANG PASIR', 'SELANGOR', '42700'],['PEKAN SIMPANG MORIB', 'SELANGOR', '42700'],
    ['PEKAN SUNGAI MANGGIS', 'SELANGOR', '42700'],['PEKAN SUNGAI RABA', 'SELANGOR', '42700'],['PEKAN TANJONG DUABELAS', 'SELANGOR', '42700'],['PEKAN TELOK DATOK', 'SELANGOR', '42700'],['PEKAN TONGKAH', 'SELANGOR', '42700'],
    ['PEKAN TELOK', 'SELANGOR', '42700'],['API-API', 'SELANGOR', '45700'],['IJOK', 'SELANGOR', '45609'],['JERAM', 'SELANGOR', '45800'],['KUALA SELANGOR', 'SELANGOR', '45000'],['PASANGAN', 'SELANGOR', '45600'],['TANJONG KARANG', 'SELANGOR', '45500'],
    ['BESTARI JAYA', 'SELANGOR', '45600'],['BANDAR KUALA SELANGOR', 'SELANGOR', '45000'],['BANDAR TANJONG KARANG', 'SELANGOR', '45500'],['PEKAN ASAM JAWA', 'SELANGOR', '45700'],['PEKAN BUKIT ROTAN', 'SELANGOR', '45620'],['PEKAN JERAM', 'SELANGOR', '45800'],
    ['PEKAN KAMPONG KUANTAN', 'SELANGOR', '45600'],['PEKAN KUALA SUNGAI BULOH', 'SELANGOR', '45800'],['PEKAN PASIR PENAMPANG', 'SELANGOR', '45000'],['PEKAN SIMPANG TIGA', 'SELANGOR', '45600'],['PEKAN TANJONG KARANG', 'SELANGOR', '45500'],
    ['PEKAN BUKTI BELIMBING', 'SELANGOR', '45000'],['PEKAN BUKIT TALANG', 'SELANGOR', '45500'],['PEKAN KAMBUNG BARU HULU TIRAM BURUK', 'SELANGOR', '45500'],['PEKAN PARTI MAHANG', 'SELANGOR', '45800'],['PEKAN SIMPANG TIGA IJOK', 'SELANGOR', '45700'],
    ['PEKAN SUNGAI SEMBILANG', 'SELANGOR', '45800'],['PEKAN TAMAN PKNS', 'SELANGOR', '45000'],['PEKAN TAMBAK JAWA', 'SELANGOR', '45600'],['PEKAN BESTARI JAYA', 'SELANGOR', '45600'],['BAGAN NAKHODA OMAR', 'SELANGOR', '45200'],
    ['PANCANG BEDENA', 'SELANGOR', '45300'],['PASIR PANJANG', 'SELANGOR', '45400'],['SABAK', 'SELANGOR', '45200'],['SUNGAI PANJANG', 'SELANGOR', '45300'],['PEKAN BAGAN TERAP', 'SELANGOR', '45209'],['PEKAN PARIT ENAM', 'SELANGOR', '45300'],
    ['PEKAN PARIT SEMBILAN', 'SELANGOR', '45300'],['PEKAN SEKINCHAN', 'SELANGOR', '45400'],['PEKAN SABAK', 'SELANGOR', '45200'],['PEKAN SUNGAI AIR TAWAR', 'SELANGOR', '45200'],['PEKAN SUNGAI SEPINTAS', 'SELANGOR', '45200'],
    ['PEKAN BAGAN NAKHODA OMAR', 'SELANGOR', '45200'],['PEKAN PARIT BARU', 'SELANGOR', '45400'],['PEKAN PASIR PANJANG', 'SELANGOR', '45400'],['PEKAN SEKINCHAN SITE A', 'SELANGOR', '45400'],['PEKAN SUNGAI BESAR', 'SELANGOR', '45300'],
    ['PEKAN SUNGAI HAJI DORANI', 'SELANGOR', '45300'],['PEKAN SUNGAI NIBONG', 'SELANGOR', '45400'],['BANDAR CHERAS', 'SELANGOR', '43200'],['BANDAR ULU LANGAT', 'SELANGOR', '43200'],['BANDAR BALAKONG', 'SELANGOR', '43300'],
    ['BANDAR BATU 9, CHERAS', 'SELANGOR', '43200'],['BERANANG', 'SELANGOR', '43700'],['HULU LANGAT', 'SELANGOR', '43100'],['HULU SEMENYIH', 'SELANGOR', '43500'],['KAJANG', 'SELANGOR', '43000'],['SEMENYIH', 'SELANGOR', '43500'],
    ['BANDAR KAJANG', 'SELANGOR', '43000'],['BANDAR SEMENYIH', 'SELANGOR', '43500'],['BANDAR AMPANG', 'SELANGOR', '68000'],['BANDAR BATU 18, SEMENYIH', 'SELANGOR', '43100'],['BANDAR BATU 26, BERANANG', 'SELANGOR', '43700'],
    ['PEKAN BERANANG', 'SELANGOR', '43700'],['PEKAN KACAU', 'SELANGOR', '43500'],['PEKAN TARUN', 'SELANGOR', '43500'],['PEKAN BANGI LAMA', 'SELANGOR', '43000'],['PEKAN BATU 23, SUNGAI LALANG', 'SELANGOR', '43500'],
    ['PEKAN BATU 26, BERANANG', 'SELANGOR', '43700'],['PEKAN BUKIT SUNGAI RAYA', 'SELANGOR', '43200'],['PEKAN CHERAS', 'SELANGOR', '43200'],['PEKAN DESA RAYA', 'SELANGOR', '43100'],['PEKAN DUSUN TUA ULU LANGAT', 'SELANGOR', '43100'],
    ['PEKAN KAJANG', 'SELANGOR', '43000'],['PEKAN KAMPONG PASIR, BATU 14, SEMENYIH', 'SELANGOR', '43000'],['PEKAN KAMPONG SUNGAI TANGKAS', 'SELANGOR', '43000'],['PEKAN RUMAH MURAH SUNGAI LUI', 'SELANGOR', '43100'],
    ['PEKAN SEMENYIH', 'SELANGOR', '43500'],['PEKAN SIMPANG BALAK', 'SELANGOR', '43000'],['PEKAN SRI NANDING', 'SELANGOR', '43100'],['PEKAN SUNGAI KEMBONG BERANANG', 'SELANGOR', '43700'],['PEKAN SUNGAI LUI', 'SELANGOR', '43100'],
    ['PEKAN SUNGAI MAKAU', 'SELANGOR', '43100'],['BULUH TELOR', 'SELANGOR', '44000'],['ULU YAM', 'SELANGOR', '44300'],['PERETAK', 'SELANGOR', '44000'],['SUNGAI GUMUT', 'SELANGOR', '44000'],['SUNGAI TINGGI', 'SELANGOR', '44200'],
    ['BANDAR ULU YAM', 'SELANGOR', '44300'],['BANDAR ULU YAM BAHARU', 'SELANGOR', '44300'],['BANDAR KUALA KUBU BAHARU', 'SELANGOR', '44000'],['BANDAR SUNGAI CHIK', 'SELANGOR', '44300'],['BATANG KALI', 'SELANGOR', '44300'],
    ['AMPANG PECHAH', 'SELANGOR', '44000'],['ULU BERNAM', 'SELANGOR', '36500'],['KALUMPANG', 'SELANGOR', '44100'],['KERLING', 'SELANGOR', '44100'],['KUALA KALUMPANG', 'SELANGOR', '44100'],['RASA', 'SELANGOR', '44200'],
    ['SERENDAH', 'SELANGOR', '48200'],['BANDAR KALUMPANG', 'SELANGOR', '44100'],['BANDAR RASA', 'SELANGOR', '44200'],['BANDAR SERENDAH', 'SELANGOR', '48200'],['BANDAR BATANG KALI', 'SELANGOR', '44300'],
    ['BANDAR ULU BERNAM I', 'SELANGOR', '36500'],['BANDAR ULU BERNAM II', 'SELANGOR', '36500'],['PEKAN KERLING', 'SELANGOR', '44100'],['PEKAN PERETAK', 'SELANGOR', '44000'],['PEKAN SIMPANG SUNGAI CHOH', 'SELANGOR', '48000'],
    ['BANDAR SHAH ALAM', 'SELANGOR', '40250'],['BANDAR DAMANSARA', 'SELANGOR', '46350'],['BANDAR GLENMARIE', 'SELANGOR', '40250'],['BANDAR PETALING JAYA SELATAN', 'SELANGOR', '46000'],['BANDAR SRI DAMANSARA', 'SELANGOR', '52200'],
    ['BUKIT RAJA', 'SELANGOR', '40170'],['DAMANSARA', 'SELANGOR', '52200'],['SUNGAI BULOH', 'SELANGOR', '47000'],['BANDAR SAUJANA', 'SELANGOR', '47000'],['BANDAR SUBANG JAYA', 'SELANGOR', '47500'],['BANDAR SUNWAY', 'SELANGOR', '47500'],
    ['PEKAN BATU TIGA', 'SELANGOR', '40150'],['PEKAN MERBAU SEMPAK', 'SELANGOR', '40160'],['PEKAN PUCHONG', 'SELANGOR', '47100'],['PEKAN SERDANG', 'SELANGOR', '43300'],['PEKAN SUNGAI BULOH', 'SELANGOR', '47000'],
    ['PEKAN CEMPAKA', 'SELANGOR', '47301'],['PEKAN COUNTRY HEIGHT', 'SELANGOR', '43000'],['PEKAN DESA PUCHONG', 'SELANGOR', '47150'],['PEKAN HICOM', 'SELANGOR', '40150'],['PEKAN KAYU ARA', 'SELANGOR', '47400'],
    ['PEKAN KINRARA', 'SELANGOR', '47180'],['PEKAN BARU HICOM', 'SELANGOR', '40400'],['PEKAN BARU SUBANG', 'SELANGOR', '47200'],['PEKAN BARU SUNGAI BESI', 'SELANGOR', '43300'],['PEKAN BARU SUNGAI BULOH', 'SELANGOR', '40160'],
    ['PEKAN BARU PENAGA', 'SELANGOR', '43400'],['PEKAN PUCHONG JAYA', 'SELANGOR', '47170'],['PEKAN PUCHONG PERDANA', 'SELANGOR', '47150'],['PEKAN SUBANG', 'SELANGOR', '40150'],['PEKAN SUBANG JAYA', 'SELANGOR', '47500'],
    ['BANDAR GOMBAK SETIA', 'SELANGOR', '53100'],['BANDAR ULU KELANG', 'SELANGOR', '68000'],['BANDAR KEPONG', 'SELANGOR', '52100'],['BANDAR KUNDANG', 'SELANGOR', '48050'],['BANDAR SUNGAI PUSU', 'SELANGOR', '53000'],
    ['RAWANG', 'SELANGOR', '48000'],['BANDAR BATU ARANG', 'SELANGOR', '48000'],['BANDAR RAWANG', 'SELANGOR', '48000'],['BANDAR SELAYANG', 'SELANGOR', '68100'],['BANDAR SUNGAI BULOH', 'SELANGOR', '47000'],
    ['PEKAN BATU 20', 'SELANGOR', '48200'],['PEKAN KUANG', 'SELANGOR', '47000'],['PEKAN MIMALAND', 'SELANGOR', '53100'],['PEKAN PENGKALAN KUNDANG', 'SELANGOR', '48020'],['PEKAN SUNGAI BULOH', 'SELANGOR', '47000'],
    ['PEKAN TEMPLER', 'SELANGOR', '48000'],['DENGKIL', 'SELANGOR', '43800'],['SEPANG', 'SELANGOR', '43900'],['BANDAR SEPANG', 'SELANGOR', '43950'],['BANDAR BARU BANGI', 'SELANGOR', '43650'],
    ['BANDAR BARU SALAK TINGGI', 'SELANGOR', '43900'],['BANDAR LAPANGAN TERBANG ANTARABANGSA SEPANG', 'SELANGOR', '43900'],['BANDAR SUNGAI MERAB', 'SELANGOR', '43000'],['PEKAN DENGKIL', 'SELANGOR', '43800'],
    ['PEKAN SALAK', 'SELANGOR', '43900'],['PEKAN SUNGAI PELEK', 'SELANGOR', '43950'],['PEKAN BATU 1 SEPANG', 'SELANGOR', '43900'],['PEKAN BUKIT BISA', 'SELANGOR', '62000'],['PEKAN BUKIT PRANG', 'SELANGOR', '43800'],
    ['PEKAN DATO BAKAR BAGINDA', 'SELANGOR', '43000'],['PEKAN BARU SALAK TINGGI', 'SELANGOR', '43900'],['PEKAN SUNGAI MERAB', 'SELANGOR', '43000'],['PEKAN TANJUNG MAS', 'SELANGOR', '43900']
]

for mukim in selangor_mukim_district_postcode_null:
    mukim_district_state['postcode']=np.where((mukim_district_state['mukim']==mukim[0])&(mukim_district_state['state']==mukim[1]),mukim[2],mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='SELANGOR')]
#mukim_district_state.isnull().sum()

,postcode,location,district,state,mukim


In [139]:
kl_mukim_district_postcode_null = \
[
    ['BATU', 'WP KUALA LUMPUR', '52100'],['CHERAS', 'WP KUALA LUMPUR', '56000'],['PETALING', 'WP KUALA LUMPUR', '46675'],['BANDAR PETALING JAYA', 'WP KUALA LUMPUR', '46200'],['ULU KELANG', 'WP KUALA LUMPUR', '68000'],
    ['SETAPAK', 'WP KUALA LUMPUR', '53300'],['BANDAR KUALA LUMPUR', 'WP KUALA LUMPUR', '50450'],['KUALA LUMPUR', 'WP KUALA LUMPUR', '50450'],['PEKAN BATU', 'WP KUALA LUMPUR', '51200'],['AMPANG', 'WP KUALA LUMPUR', '68000'],
    ['PEKAN SUNGAI PENCHALA', 'WP KUALA LUMPUR', '68000'],['BANDAR BANDAR BAHARU SUNGAI BESI', 'WP KUALA LUMPUR', '57000'],['PEKAN BATU CAVES', 'WP KUALA LUMPUR', '68100'],['PEKAN KEPONG', 'WP KUALA LUMPUR', '52100'],
    ['PEKAN KUALA PAUH', 'WP KUALA LUMPUR', '50460'],['PEKAN PETALING', 'WP KUALA LUMPUR', '50603'],['PEKAN SALAK SOUTH', 'WP KUALA LUMPUR', '57100']

]

for mukim in kl_mukim_district_postcode_null:
    mukim_district_state['postcode']=np.where((mukim_district_state['mukim']==mukim[0])&(mukim_district_state['state']==mukim[1]),mukim[2],mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='WP KUALA LUMPUR')]
#mukim_district_state.isnull().sum()

,postcode,location,district,state,mukim


In [160]:
sarawak_mukim_district_postcode_null = \
[
    ['PUEH LAND', 'KUCHING', '94100'],['STUNGKOR', 'KUCHING', '94500'],['SAMPADI', 'KUCHING', '93070'],['JAGOI', 'KUCHING', '94000'],['SENGGI POAK', 'KUCHING', '94000'],
    ['SALAK', 'KUCHING', '93050'],['PANGKALAN AMPAT', 'KUCHING', '94700'],['KUCHING UTARA', 'KUCHING', '93050'],['KUCHING TENGAH', 'KUCHING', '93050'],['BANDAR KUCHING', 'KUCHING', '93100'],
    ['SENTAH-SEGU', 'KUCHING', '94200'],['BANDAR BATU KAWA', 'KUCHING', '93250'],['BANDAR SUNGAI TENGAH', 'KUCHING', '93050'],['BANDAR SUNGAI KITANG', 'KUCHING', '93250'],['BANDAR BATU KITANG', 'KUCHING', '93250'],
    ['BATU 15, JALAN SENGGANG', 'KUCHING', '94500'],['175, JALAN SENGGANG', 'KUCHING', '94500'],['BANDAR JALAN BATU 19, JALAN SENGGANG', 'KUCHING', '94500'],['BANDAR BATU 24, JALAN SENGGANG', 'KUCHING', '94500'],
    ['BANDAR PANGKALAN KUT', 'KUCHING', '94700'],['BANDAR BELIONG', 'KUCHING', '94300'],['BANDAR BAKO', 'KUCHING', '93010'],['BANDAR JANGKAR', 'KUCHING', '94500'],['BANDAR RAMBUNGAN', 'KUCHING', '94500'],
    ['BANDAR STUNKOR', 'KUCHING', '94500'],['BANDAR KRANJI', 'KUCHING', '94750'],['BANDAR SINIWAN', 'KUCHING', '94000'],['BANDAR MUSI', 'KUCHING', '94000'],['BANDAR TAI TON', 'KUCHING', '94000'],
    ['BANDAR BIDI', 'KUCHING', '94000'],['BANDAR PANGKALAN TEBANG', 'KUCHING', '94000'],['BANDAR PEJIRU', 'KUCHING', '94000'],['BANDAR BARATOK', 'KUCHING', '94200'],['BANDAR TERBAT', 'KUCHING', '94700'],
    ['BANDAR JALAN BATU 17', 'KUCHING', '95000'],['BANDAR JALAN BATU 17, JALAN SENGGANG', 'KUCHING', '95000'],['GADING LUNDU', 'KUCHING', '94500'],['MATANG', 'KUCHING', '93050'],['MUARA TEBAS', 'KUCHING', '93010'],
    ['BATU 8, JALAN MATANG', 'KUCHING', '93050'],['BANDAR SEMATANG', 'KUCHING', '94500'],['BANDAR LUNDU', 'KUCHING', '94500'],['BANDAR PAKU', 'KUCHING', '94000'],['BANDAR JAMBUSAN', 'KUCHING', '94000'],
    ['BANDAR BAU', 'KUCHING', '94000'],['BANDAR BUSO', 'KUCHING', '94000'],['BANDAR TUNDONG', 'KUCHING', '94000'],['BANDAR KROKONG', 'KUCHING', '94000'],['BANDAR TIANG BEKAP', 'KUCHING', '93010'],
    ['BANDAR TAPAH', 'KUCHING', '94200'],['BANDAR SIBURAN', 'KUCHING', '94200'],['UNDUP', 'SRI AMAN', '95000'],['KLAUH', 'SRI AMAN', '95000'],['BIJAT', 'SRI AMAN', '95000'],['SKARANG', 'SRI AMAN', '95000'],
    ['KERANGGAS', 'SRI AMAN', '95007'],['MARUP', 'SRI AMAN', '95800'],['LAMANAK', 'SRI AMAN', '95000'],['BUKIT BESAI', 'SRI AMAN', '95900'],['AI ENGKARI', 'SRI AMAN', '95900'],['LESONG', 'SRI AMAN', '95000'],
    ['SELANJANG', 'SRI AMAN', '95000'],['SILANTEK', 'SRI AMAN', '95000'],['SIRNANGGANG', 'SRI AMAN', '95000'],['BANDAR LINGGA', 'SRI AMAN', '94900'],['BANDAR LUBOK ANTU', 'SRI AMAN', '95900'],
    ['BANDAR ENGKILILI', 'SRI AMAN', '95800'],['BANDAR BATU LINTANG', 'SRI AMAN', '95000'],['BANDAR BANTING', 'SRI AMAN', '95000'],['BANDAR PANTU', 'SRI AMAN', '95000'],['BANDAR BAKONG', 'SRI AMAN', '95000'],
    ['BANDAR UNDUP', 'SRI AMAN', '95000'],['BANDAR SKRANG', 'SRI AMAN', '95000'],['BANDAR MELUGU', 'SRI AMAN', '95000'],['BANDAR SABU', 'SRI AMAN', '95000'],['UNDUP', 'SRI AMAN', '95000'],
    ['KLAUH', 'SRI AMAN', '95000'],['BIJAT', 'SRI AMAN', '95000'],['SKARANG', 'SRI AMAN', '95000'],['KERANGGAS', 'SRI AMAN', '95000'],['MARUP', 'SRI AMAN', '95800'],['LAMANAK', 'SRI AMAN', '95000'],
    ['BUKIT BESAI', 'SRI AMAN', '95000'],['AI ENGKARI', 'SRI AMAN', '95000'],['LESONG', 'SRI AMAN', '95000'],['SELANJANG', 'SRI AMAN', '95000'],['SILANTEK', 'SRI AMAN', '95000'],['SIRNANGGANG', 'SRI AMAN', '95000'],
    ['BANDAR LINGGA', 'SRI AMAN', '94900'],['BANDAR LUBOK ANTU', 'SRI AMAN', '95900'],['BANDAR ENGKILILI', 'SRI AMAN', '95800'],['BANDAR BATU LINTANG', 'SRI AMAN', '95000'],['BANDAR BANTING', 'SRI AMAN', '95000'],
    ['BANDAR PANTU', 'SRI AMAN', '95000'],['BANDAR BAKONG', 'SRI AMAN', '95000'],['BANDAR UNDUP', 'SRI AMAN', '95000'],['BANDAR SKRANG', 'SRI AMAN', '95000'],['BANDAR MELUGU', 'SRI AMAN', '95000'],
    ['BANDAR SABU', 'SRI AMAN', '95000'],['SEDUAN', 'SIBU', '96000'],['ENGKILO', 'SIBU', '96000'],['PASAI-SIONG', 'SIBU', '96000'],['ASSAN', 'SIBU', '96000'],['MENYAN', 'SIBU', '96000'],['KABANG', 'SIBU', '96000'],
    ['LUKUT', 'SIBU', '96000'],['MAPAI', 'SIBU', '96000'],  # Not officially listed, fallback assigned
    ['MAROH', 'SIBU', '96000'],['SPALI', 'SIBU', '96000'],  # Unverified, fallback assigned
    ['QYA-DALAT', 'SIBU', '96000'],  # Name likely incorrect, fallback used
    ['SPAPA', 'SIBU', '96000'],  # Unverified
    ['PAKU', 'SIBU', '96000'],  # Could overlap with Sri Aman’s PAKU, use with caution
    ['LALAI', 'SIBU', '96000'],  # Unverified
    ['MUKAH', 'SIBU', '96000'],  # Mukah is a separate division, but some listings show 96000
    ['GIGIS', 'SIBU', '96000'],  # Not found, fallback
    ['SELANGAU', 'SIBU', '96000'],['BALINGAN', 'SIBU', '96000'],['ARIP', 'SIBU', '96000'],['PELUNGAU', 'SIBU', '96000'],['BAWAN', 'SIBU', '96000'],['BULOH', 'SIBU', '96000'],['BANDAR SIBU', 'SIBU', '96000'],
    ['BANDAR SENGEI MERAH', 'SIBU', '96000'],['BANDAR TEKU', 'SIBU', '96000'],['BANDAR DURIN', 'SIBU', '96000'],['BANDAR KANOWIT', 'SIBU', '96000'],
    ['BANDAR DAP', 'SIBU', '96000'],  # Possible name ambiguity
    ['BANDAR MACHAN', 'SIBU', '96000'],['BANDAR NGEMAH', 'SIBU', '96000'],['BANDAR SENGAYAN', 'SIBU', '96000'],['BANDAR SIBINTEK', 'SIBU', '96000'],['KONSESI MIRI', 'MIRI', '98000'],['BANDAR LUTONG', 'MIRI', '98100'],
    ['BANDAR BAZAR JALAN RIAM', 'MIRI', '98000'],['KUALA BARAM', 'MIRI', '98000'],['LAMBIR', 'MIRI', '98000'],['BANDAR KUALA NYABOR', 'MIRI', '98000'],  # Not explicitly listed, fallback used
    ['BAREO', 'MIRI', '98000'],  # Unlisted, fallback assigned
    ['APOH', 'MIRI', '98000'],  # Not found, fallback used
    ['LIO MATOH', 'MIRI', '98000'],  # Remote area, fallback used
    ['SILAT', 'MIRI', '98000'],  # Not listed, fallback used
    ['TUTOH', 'MIRI', '98000'],  # Not listed, fallback used
    ['PATAH', 'MIRI', '98000'],  # Not listed, fallback used
    ['LEPU LEJU', 'MIRI', '98000'],  # Not listed, fallback used
    ['DANAU', 'LIMBANG', '98700'],['PANARUAN', 'LIMBANG', '98700'],['TRUSAN', 'LIMBANG', '98850'],['LAWAS', 'LIMBANG', '98850'],['MERAPOK', 'LIMBANG', '98850'],['LIMBANG', 'LIMBANG', '98700'],
    ['NANGA MEDAMIT', 'LIMBANG', '98750'],['UKONG', 'LIMBANG', '98700'],['BANDAR SUNDAR', 'LIMBANG', '98700'],['SUNGAI ADANG', 'LIMBANG', '98700'],['LONG NAPIR', 'LIMBANG', '98700'],['SUNGAI ADDANG', 'LIMBANG', '98700'],
    ['TENGOA-SUKANG', 'LIMBANG', '98700'],['LONG NERAPAP', 'LIMBANG', '98700'],['LONG SEMADO', 'LIMBANG', '98700'],['BAKALALAN', 'LIMBANG', '98700'],['BATU LAWI', 'LIMBANG', '98700'],['SERENDANG', 'SARIKEI', '96100'],
    ['MARADONG', 'SARIKEI', '96100'],['TULAI', 'SARIKEI', '96100'],['SARIKEI', 'SARIKEI', '96100'],['BUAN', 'SARIKEI', '96100'],['SARE', 'SARIKEI', '96100'],['PEDANUM', 'SARIKEI', '96100'],['MELURUN', 'SARIKEI', '96100'],
    ['JIKANG', 'SARIKEI', '96100'],['BANDAR BINATANG', 'SARIKEI', '96100'],['JULAU', 'SARIKEI', '96600'],['BANDAR PAKAN', 'SARIKEI', '96100'],['BANDAR SELALANG', 'SARIKEI', '96100'],['GUNUNG AYER', 'SARIKEI', '96100'],
    ['BINATANG', 'SARIKEI', '96100'],['KATIBAS', 'KAPIT', '96850'],['IBAU', 'KAPIT', '96800'],['MENUAN', 'KAPIT', '96800'],['SUAU', 'KAPIT', '96800'],['OYAN', 'KAPIT', '96800'],['BANING', 'KAPIT', '96800'],
    ['MAJAU', 'KAPIT', '96800'],['MENRAL', 'KAPIT', '96800'],['METAH', 'KAPIT', '96800'],['RIRONG', 'KAPIT', '96800'],['MAMU', 'KAPIT', '96800'],['ANGKUAH', 'KAPIT', '96800'],['PELAGUS', 'KAPIT', '96800'],['BANDAR KAPIT', 'KAPIT', '96800'],
    ['BANDAR SONG', 'KAPIT', '96850'],['BANGKIT', 'KAPIT', '96850'],['BATU LAGA', 'KAPIT', '96800'],['PELANDUK', 'KAPIT', '96800'],['ENTEMU', 'KAPIT', '96800'],['MENGIONG', 'KAPIT', '96800'],['SERANI', 'KAPIT', '96800'],['BALUI', 'KAPIT', '96800'],
    ['KUMBONG', 'KAPIT', '96800'],['MURUM', 'KAPIT', '96800'],['PUNAN', 'KAPIT', '96800'],['DANUM', 'KAPIT', '96800'],['BANDAR MALUDAM', 'BETONG', '94950'],['BINTULU', 'BINTULU', '97000'],['KEMENA', 'BINTULU', '97000'],['SEBAUH', 'BINTULU', '97100'],
    ['BANDAR LANANG', 'BINTULU', '97000'],['BANDAR PANDAN', 'BINTULU', '97100'],['BANDAR TUBAU', 'BINTULU', '97100'],['SELEZU', 'BINTULU', '97100'],['BATU KAPAL', 'BINTULU', '97000'],['RASAN', 'BINTULU', '97000'],['PANDAN', 'BINTULU', '97100'],
    ['BANDAR KUALA TATAU', 'BINTULU', '97200'],['BANDAR TATAU', 'BINTULU', '97200'],['BUAN LAN', 'BINTULU', '97000'],['SANGAN', 'BINTULU', '97200'],['ANAP', 'BINTULU', '97200'],['KAKUS', 'BINTULU', '97200'],['JELALONG', 'BINTULU', '97200'],
    ['BINI', 'BINTULU', '97000'],['MENUKU', 'SAMARAHAN', '94300'],['SAMARAHAN', 'SAMARAHAN', '94300'],['MELIKIN LAND', 'SAMARAHAN', '94700'],['PUNDA-SABAL', 'SAMARAHAN', '94700'],['BANDAR SUNGAI MERAH', 'SAMARAHAN', '94700'],
    ['BANDAR SUNAGAI MERANG', 'SAMARAHAN', '94700'],['BANDAR SUNGAI PALAH', 'SAMARAHAN', '94700'],['BANDAR TEDEBU', 'SAMARAHAN', '94760'],['BANDAR BALAI RINGIN', 'SAMARAHAN', '94700'],['BANDAR TAMBIRAT', 'SAMARAHAN', '94600'],
    ['BANDAR SIMUJAN', 'SAMARAHAN', '94700'],['BANDAR TEBELU', 'SAMARAHAN', '94850'],['BANDAR TERBAT', 'SAMARAHAN', '94700'],['KAYAN', 'SAMARAHAN', '94700'],['MUARA TUANG', 'SAMARAHAN', '94300'],['BUKAR-SADONG', 'SAMARAHAN', '94700'],
    ['BANDAR MUARA TUANG', 'SAMARAHAN', '94300'],['BANDAR GEDONG', 'SAMARAHAN', '94700'],['SEDILU-GEDONG', 'SAMARAHAN', '94800'],['SADONG', 'SAMARAHAN', '94600'],['LESONG', 'SAMARAHAN', '94300'],['SUNGAI KEDUP', 'SAMARAHAN', '94300'],
    ['SEBANGAN-KEPAYAN', 'SAMARAHAN', '94800'],['BANDAR SEBUYAU', 'SAMARAHAN', '94850'],['BATU 29, JALAN SIMANGGANG', 'SAMARAHAN', '94300'],['BATU 32, JALAN SIMANGGANG', 'SAMARAHAN', '94300'],['BATU 34+, JALAN SIMANGGANG', 'SAMARAHAN', '94300'],
    ['LAWAS', 'LAWAS', '98850'],['SUNGAI BANGAT', 'LAWAS', '98850'],['MERAPOK', 'LAWAS', '98850'],['TRUSAN', 'LAWAS', '98850'],['LONG TUMA', 'LAWAS', '98850'],['LONG SUKANG', 'LAWAS', '98850'],['LONG SEMADOH', 'LAWAS', '98850'],['LONG LUPING', 'LAWAS', '98850'],
    ['LONG TENGOA', 'LAWAS', '98850'],['LONG TUKON', 'LAWAS', '98850'],['LONG SEBANGAN', 'LAWAS', '98850'],['PUNANG', 'LAWAS', '98850'],['AWAT-AWAT', 'LAWAS', '98850'],['SUNDAR', 'LAWAS', '98850'],['LUAGAN', 'LAWAS', '98850'],['SIANG-SIANG', 'LAWAS', '98850'],
    ['TANG LAPADAN', 'LAWAS', '98850'],['ARU BARU', 'LAWAS', '98850'],['KAMPUNG LINTANG', 'LAWAS', '98850'],['KAMPUNG SEBERANG', 'LAWAS', '98850'],['BATU LIMA', 'LAWAS', '98850'],['BELIPAT', 'LAWAS', '98850'],['KUALA LAWAS', 'LAWAS', '98850'],['KERANGAN', 'LAWAS', '98850'],
    ['BA’KELALAN', 'LAWAS', '98850'],['BANDAR SERIAN', 'SAMARAHAN', '94700'],['BANDAR TEBAKANG', 'SAMARAHAN', '94700'],['BANDAR MUARA MONGKOS', 'SAMARAHAN', '94750'],['BANDAR SEBANGAN', 'SAMARAHAN', '94800']
]

for mukim in sarawak_mukim_district_postcode_null:
    mukim_district_state['postcode']=np.where((mukim_district_state['mukim']==mukim[0])&(mukim_district_state['district']==mukim[1]),mukim[2],mukim_district_state['postcode'])


mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='SARAWAK')][[ 'district', 'state', 'mukim']]

#mukim_district_state.isnull().sum()

,district,state,mukim


In [141]:
kelantan_mukim_district_postcode_null = \
[
    ['ENGGONG', 'KUALA KRAI', '18000'],['GAJAH', 'KUALA KRAI', '18000'],['KANDEK', 'KUALA KRAI', '18000'],['KENOR', 'KUALA KRAI', '18000'],['KUALA GERIS', 'KUALA KRAI', '18200'],['KUALA KRAI', 'KUALA KRAI', '18000'],['KUALA NAL', 'KUALA KRAI', '18000'],
    ['KUALA PERGAU', 'KUALA KRAI', '18200'],['KUALA STONG', 'KUALA KRAI', '18200'],['MAMBONG', 'KUALA KRAI', '18000'],['MANJOR', 'KUALA KRAI', '18000'],['TELEKONG', 'KUALA KRAI', '18000'],['BANDAR KUALA KRAI', 'KUALA KRAI', '18000'],['AUR DURI', 'KOTA BHARU', '15100'],
    ['BADAK MATI', 'KOTA BHARU', '16150'],['BADAK', 'KOTA BHARU', '16150'],['BADANG', 'KOTA BHARU', '15350'],['BANGGU', 'KOTA BHARU', '16150'],['BAUNG', 'KOTA BHARU', '16100'],['BAYANG', 'KOTA BHARU', '15200'],['BETA HULU', 'KOTA BHARU', '16450'],  # Falls under Ketereh, within KB district
    ['BETING', 'KOTA BHARU', '16100'],['BIAH', 'KOTA BHARU', '16010'],['BINJAI', 'KOTA BHARU', '16450'],['BULOH POH', 'KOTA BHARU', '15200'],['BUT', 'KOTA BHARU', '15100'],['CHEKLI', 'KOTA BHARU', '15100'],['CHICHA', 'KOTA BHARU', '16100'],['DAL', 'KOTA BHARU', '16400'],
    ['DEMIT', 'KOTA BHARU', '16100'],['DUSON RENDAH', 'KOTA BHARU', '16100'],['GUNTONG', 'KOTA BHARU', '16450'],['JELUTONG', 'KOTA BHARU', '16100'],['KARANG', 'KOTA BHARU', '15100'],['KAMPONG SIREH', 'KOTA BHARU', '15000'],['KIJANG', 'KOTA BHARU', '16100'],
    ['KEMUMIN', 'KOTA BHARU', '16100'],['KENALI', 'KOTA BHARU', '16100'],['KOH', 'KOTA BHARU', '15050'],['KOTA', 'KOTA BHARU', '15000'],['LEMBU', 'KOTA BHARU', '15100'],['LUBOK PUKOL', 'KOTA BHARU', '16100'],['LUNDANG', 'KOTA BHARU', '15150'],['LUNDANG PAKU', 'KOTA BHARU', '15150'],
    ['MAHANG BARAT', 'KOTA BHARU', '16450'],['MAHANG TIMOR', 'KOTA BHARU', '16450'],['MENTUAN', 'KOTA BHARU', '16100'],['MELOR', 'KOTA BHARU', '16400'],['PADANG BONGOR', 'KOTA BHARU', '16100'],['PADANG ENGGANG', 'KOTA BHARU', '15100'],['PADANG GARONG', 'KOTA BHARU', '15050'],
    ['PADANG LEBAN', 'KOTA BHARU', '16100'],['PADANG RAJA', 'KOTA BHARU', '15100'],['PADANG SAKAR', 'KOTA BHARU', '15100'],['PADANG TENGAH', 'KOTA BHARU', '15100'],['PANGKAL PISANG', 'KOTA BHARU', '16100'],['PARIT', 'KOTA BHARU', '15000'],['PASIR HA', 'KOTA BHARU', '15000'],
    ['PATEK', 'KOTA BHARU', '16100'],['PAUH', 'KOTA BHARU', '16450'],['PAYA', 'KOTA BHARU', '15050'],['PENDEK', 'KOTA BHARU', '16150'],['PANGKALAN CHEPA', 'KOTA BHARU', '16100'],['PERINGAT', 'KOTA BHARU', '16400'],['PEROL', 'KOTA BHARU', '15100'],
    ['PINTU GANG', 'KOTA BHARU', '15100'],['PULAU', 'KOTA BHARU', '15050'],['PULAU BELANGA', 'KOTA BHARU', '15050'],['PULAU KUNDOR', 'KOTA BHARU', '16100'],['PULAU PANJANG', 'KOTA BHARU', '15050'],['PULAU PISANG', 'KOTA BHARU', '16100'],['SEMUT API', 'KOTA BHARU', '15100'],
    ['SERING', 'KOTA BHARU', '16400'],['SETERPA', 'KOTA BHARU', '16400'],['TANGJONG CHAT', 'KOTA BHARU', '15000'],['TELOK', 'KOTA BHARU', '15050'],['TELOK BHARU', 'KOTA BHARU', '15050'],['TELOK KITANG', 'KOTA BHARU', '15050'],['TIONG', 'KOTA BHARU', '15100'],
    ['TOK KU', 'KOTA BHARU', '15050'],['WAKAF STAN', 'KOTA BHARU', '16100'],['BANDAR KOTA BHARU', 'KOTA BHARU', '15000']
]

for mukim in kelantan_mukim_district_postcode_null:
    mukim_district_state['postcode']=np.where((mukim_district_state['mukim']==mukim[0])&(mukim_district_state['district']==mukim[1]),mukim[2],mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode']=='')&(mukim_district_state['state']=='KELANTAN')][[ 'district', 'state', 'mukim']]
#mukim_district_state.isnull().sum()


,district,state,mukim


In [142]:
melaka_mukim_district_postcode_null = \
[
    ['ALAI', 'MELAKA TENGAH', '75460'],['BACHANG', 'MELAKA TENGAH', '75350'],['BERTAM', 'MELAKA TENGAH', '76450'],['CHENG', 'MELAKA TENGAH', '75250'],['KRUBONG', 'MELAKA TENGAH', '75260'],['PADANG TEMU', 'MELAKA TENGAH', '75050'],['PRINGGIT', 'MELAKA TENGAH', '75400'],
    ['TANGGA BATU', 'MELAKA TENGAH', '76400'],['TANJONG KELING', 'MELAKA TENGAH', '76400'],['TANJONG MINYAK', 'MELAKA TENGAH', '75260'],['TELOK MAS', 'MELAKA TENGAH', '75460'],['BANDAR MELAKA', 'MELAKA TENGAH', '75400'],['AYER MOLEK', 'MELAKA TENGAH', '75460'],
    ['BALAI PANJANG', 'MELAKA TENGAH', '75250'],['BATU BERENDAM', 'MELAKA TENGAH', '75350'],['BUKIT BARU', 'MELAKA TENGAH', '75150'],['BUKIT KATIL', 'MELAKA TENGAH', '75450'],['BUKIT LINTANG', 'MELAKA TENGAH', '75460'],['BUKIT PIATU', 'MELAKA TENGAH', '75150'],
    ['BUKIT RAMBAI', 'MELAKA TENGAH', '75260'],['DUYONG', 'MELAKA TENGAH', '75460'],['UJONG PASIR', 'MELAKA TENGAH', '75050'],['KANDANG', 'MELAKA TENGAH', '75460'],['KLEBANG BESAR', 'MELAKA TENGAH', '75200'],['KLEBANG KECHIL', 'MELAKA TENGAH', '75200'],
    ['PADANG SEMABOK', 'MELAKA TENGAH', '75050'],['PAYA RUMPUT', 'MELAKA TENGAH', '76450'],['PERNU', 'MELAKA TENGAH', '75460'],['SEMABOK', 'MELAKA TENGAH', '75050'],['SUNGAI UDANG', 'MELAKA TENGAH', '76300'],['BANDAR BUKIT BARU', 'MELAKA TENGAH', '75150'],
    ['PEKAN AYER MOLEK', 'MELAKA TENGAH', '75460'],['PEKAN BATU BERENDAM', 'MELAKA TENGAH', '75350'],['PEKAN BUKIT RAMBAI', 'MELAKA TENGAH', '75260'],['PEKAN KANDANG', 'MELAKA TENGAH', '75460'],['PEKAN PAYA RUMPUT', 'MELAKA TENGAH', '76450'],
    ['PEKAN SUNGAI UDANG', 'MELAKA TENGAH', '76300'],['PEKAN TANJONG KLING', 'MELAKA TENGAH', '76400'],['PEKAN KLEBANG', 'MELAKA TENGAH', '75200'],['PEKAN TANGGA BATU', 'MELAKA TENGAH', '76400'],['AYER PANAS', 'JASIN', '77200'],['BATANG MALAKA', 'JASIN', '77500'],
    ['BUKIT SENGGEH', 'JASIN', '77100'],['CHABAU', 'JASIN', '77100'],['CHIN CHIN', 'JASIN', '77000'],['CHOHONG', 'JASIN', '77000'],['JASIN', 'JASIN', '77000'],['JUS', 'JASIN', '77000'],['KESANG', 'JASIN', '77000'],['MERLIMAU', 'JASIN', '77300'],['NYALAS', 'JASIN', '77100'],
    ['RIM', 'JASIN', '77000'],['SEBATU', 'JASIN', '77300'],['SELANDAR', 'JASIN', '77500'],['SEMPANG', 'JASIN', '77000'],['SEMUJOK', 'JASIN', '77300'],['SERKAM', 'JASIN', '77300'],['SUNGEI RAMBAI', 'JASIN', '77400'],['TEDONG', 'JASIN', '77300'],['UMBAI', 'JASIN', '77300'],
    ['BANDAR JASIN', 'JASIN', '77000'],['BANDAR MERLIMAU', 'JASIN', '77300'],['PEKAN ASAHAN', 'JASIN', '77100'],['PEKAN BATANG MALAKA', 'JASIN', '77500'],['PEKAN BEMBAN', 'JASIN', '77200'],['PEKAN CHIN CHIN', 'JASIN', '77000'],['PEKAN KESANG PAJAK', 'JASIN', '77000'],
    ['PEKAN NYALAS', 'JASIN', '77100'],['PEKAN SELANDAR', 'JASIN', '77500'],['PEKAN SEMPANG BEKOH', 'JASIN', '77000'],['PEKAN SUNGAI RAMBAI', 'JASIN', '77400'],['AYER PA\'ABAS', 'ALOR GAJAH', '78000'],['BELIMBING', 'ALOR GAJAH', '76100'],['BERINGIN', 'ALOR GAJAH', '76100'],
    ['BRISU', 'ALOR GAJAH', '78000'],['DURIAN TUNGGAL', 'ALOR GAJAH', '76100'],['GADEK', 'ALOR GAJAH', '78000'],['KELEMAK', 'ALOR GAJAH', '78000'],['KEMUNING', 'ALOR GAJAH', '78000'],['KUALA LINGGI', 'ALOR GAJAH', '78200'],['KUALA SUNGEI BARU', 'ALOR GAJAH', '78200'],
    ['LENDU', 'ALOR GAJAH', '78000'],['MACHAP', 'ALOR GAJAH', '78000'],['MASJID TANAH', 'ALOR GAJAH', '78300'],['MELAKA PINDAH', 'ALOR GAJAH', '78000'],['MELEKEK', 'ALOR GAJAH', '78000'],['PADANG SEBANG', 'ALOR GAJAH', '78000'],['PARIT MELANA', 'ALOR GAJAH', '76100'],
    ['PEGOH', 'ALOR GAJAH', '78000'],['PULAU SEBANG', 'ALOR GAJAH', '78000'],['RAMUAN CHINA BESAR', 'ALOR GAJAH', '78300'],['RAMUAN CHINA KECHIL', 'ALOR GAJAH', '78300'],['REMBIA', 'ALOR GAJAH', '78100'],['SUNGEI BARU HILIR', 'ALOR GAJAH', '78200'],
    ['SUNGEI BARU TENGAH', 'ALOR GAJAH', '78200'],['SUNGEI BARU ULU', 'ALOR GAJAH', '78200'],['SUNGEI BULOH', 'ALOR GAJAH', '78000'],['SUNGEI PETAI', 'ALOR GAJAH', '78000'],['SUNGEI SIPUT', 'ALOR GAJAH', '78000'],['TABOH NANING', 'ALOR GAJAH', '78000'],
    ['TANJONG RIMAU', 'ALOR GAJAH', '78000'],['TEBONG', 'ALOR GAJAH', '78000'],['BANDAR ALOR GAJAH', 'ALOR GAJAH', '78000'],['BANDAR MASJID TANAH', 'ALOR GAJAH', '78300'],['BANDAR PULAU SEBANG', 'ALOR GAJAH', '78000'],['PEKAN DURIAN TUNGGAL', 'ALOR GAJAH', '76100'],
    ['PEKAN KUALA SUNGAI BARU', 'ALOR GAJAH', '78200'],['PEKAN LUBOK CHINA', 'ALOR GAJAH', '78100'],['PEKAN REMBIA', 'ALOR GAJAH', '78100']
]

for mukim in melaka_mukim_district_postcode_null:
    mukim_district_state['postcode']=np.where((mukim_district_state['mukim']==mukim[0])&(mukim_district_state['district']==mukim[1]),mukim[2],mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='MELAKA')][[ 'district', 'state', 'mukim']]

#mukim_district_state.isnull().sum()

,district,state,mukim


In [143]:
n9_mukim_district_postcode_null = \
[
    ['GELAMI LEMI', 'JELEBU', '71600'],['KENABOI', 'JELEBU', '71650'],['KUALA KELAWANG', 'JELEBU', '71600'],['TERIANG HILIR', 'JELEBU', '72400'],['BANDAR KUALA KELAWANG', 'JELEBU', '71600'],['PERADONG', 'JELEBU', '71600'],['PERTANG', 'JELEBU', '72300'],
    ['PEKAN KUALA KELAWANG', 'JELEBU', '71600'],['PEKAN PERTANG', 'JELEBU', '72300'],['PEKAN TITI', 'JELEBU', '71650'],['PEKAN SIMPANG DURIAN', 'JELEBU', '72400'],['PEKAN SIMPANG PERTANG', 'JELEBU', '72300'],['PEKAN SUNGAI MUNTOH', 'JELEBU', '71650'],
    ['AMPANG TINGGI', 'KUALA PILAH', '72000'],['ULU JEMPOL', 'KUALA PILAH', '72000'],['ULU MUAR', 'KUALA PILAH', '72000'],['JOHOL', 'KUALA PILAH', '73100'],['JUASSEH', 'KUALA PILAH', '72500'],['KEPIS', 'KUALA PILAH', '73100'],['LANGKAP', 'KUALA PILAH', '72000'],
    ['PARIT TINGGI', 'KUALA PILAH', '72000'],['PILAH', 'KUALA PILAH', '72000'],['SRI MENANTI', 'KUALA PILAH', '71550'],['TERACHI', 'KUALA PILAH', '72000'],['BANDAR KUALA PILAH', 'KUALA PILAH', '72000'],['PEKAN DANGI', 'KUALA PILAH', '72000'],
    ['PEKAN GUNUNG PASIR', 'KUALA PILAH', '72000'],['PEKAN JOHOL', 'KUALA PILAH', '73100'],['PEKAN PARIT TINGGI', 'KUALA PILAH', '72000'],['PEKAN SENALING', 'KUALA PILAH', '72000'],['PEKAN TANJONG IPOH', 'KUALA PILAH', '71500'],['PEKAN JUASSEH', 'KUALA PILAH', '72500'],
    ['PEKAN BUKIT GELUGOR', 'KUALA PILAH', '72500'],['PEKAN MELANG', 'KUALA PILAH', '72000'],['PEKAN AR MAWANG', 'KUALA PILAH', '72000'],['PEKAN DANGI BARU', 'KUALA PILAH', '72000'],['JIMAH', 'PORT DICKSON', '71010'],['LINGGI', 'PORT DICKSON', '71150'],
    ['PORT DICKSON', 'PORT DICKSON', '71000'],['SI RUSA', 'PORT DICKSON', '71050'],['BANDAR PORT DICKSON', 'PORT DICKSON', '71000'],['BANDAR TELUK KEMANG', 'PORT DICKSON', '71050'],['PEKAN LUKUT', 'PORT DICKSON', '71010'],['PEKAN PASIR PANJANG', 'PORT DICKSON', '71150'],
    ['PEKAN PENGKALAN KEMPAS', 'PORT DICKSON', '71150'],['PEKAN CHUAH', 'PORT DICKSON', '71960'],['PEKAN PORT DICKSON', 'PORT DICKSON', '71000'],['PEKAN TELUK KEMANG', 'PORT DICKSON', '71050'],['PEKAN BUKIT PELANDUK', 'PORT DICKSON', '71960'],
    ['PEKAN LINGGI', 'PORT DICKSON', '71150'],['PEKAN AIR KUNING', 'PORT DICKSON', '71000'],['PEKAN SUNGAI MENYALA', 'PORT DICKSON', '71050'],['PEKAN BAGAN PINANG', 'PORT DICKSON', '71000'],['PEKAN TANAH MERAH UTARA', 'PORT DICKSON', '71960'],
    ['PEKAN TANAH MERAH SELATAN', 'PORT DICKSON', '71960'],['PEKAN JEMIMA', 'PORT DICKSON', '71100'],['BATU HAMPAR', 'REMBAU', '71300'],['BONGEK', 'REMBAU', '71300'],['CHEMBONG', 'REMBAU', '71300'],['CHENGKAU', 'REMBAU', '71300'],['GADONG', 'REMBAU', '71400'],
    ['KUNDOR', 'REMBAU', '71400'],['LEGONG ILIR', 'REMBAU', '71400'],['LEGONG ULU', 'REMBAU', '71400'],['MIKU', 'REMBAU', '71400'],['NERASAU', 'REMBAU', '71400'],['PEDAS', 'REMBAU', '71300'],['PILIN', 'REMBAU', '71300'],['SELEMAK', 'REMBAU', '71400'],
    ['SEMERBOK', 'REMBAU', '71400'],['SPRI', 'REMBAU', '71400'],['TANJONG KELING', 'REMBAU', '71400'],['TITIAN BINTANGOR', 'REMBAU', '71400'],['BANDAR REMBAU', 'REMBAU', '71300'],['PEKAN KAMPONG BATU', 'REMBAU', '71300'],['PEKAN KOTA', 'REMBAU', '71300'],
    ['PEKAN LUBOK CHINA', 'REMBAU', '71400'],['PEKAN PEDAS', 'REMBAU', '71300'],['PEKAN CHEMBONG', 'REMBAU', '71300'],['PEKAN REMBAU', 'REMBAU', '71300'],['PEKAN CHENGKAU', 'REMBAU', '71300'],['PEKAN SERI KOTA', 'REMBAU', '71400'],['PEKAN SERI KENDONG', 'REMBAU', '71400'],
    ['PEKAN MERBAU SEMBILAN', 'REMBAU', '71400'],['LABU', 'SEREMBAN', '71900'],['SEREMBAN', 'SEREMBAN', '70000'],['SETUL', 'SEREMBAN', '71800'],['BANDAR SEREMBAN', 'SEREMBAN', '70000'],['BANDAR SEREMBAN UTAMA', 'SEREMBAN', '70000'],['AMPANGAN', 'SEREMBAN', '70400'],
    ['RASAH', 'SEREMBAN', '70300'],['RANTAU', 'SEREMBAN', '71200'],['PANTAI', 'SEREMBAN', '71770'],['LENGGENG', 'SEREMBAN', '71770'],['BANDAR SEREMBAN 3', 'SEREMBAN', '70300'],['BANDAR MANTIN UTAMA', 'SEREMBAN', '71700'],['BANDAR BARU ENSTEK', 'SEREMBAN', '71760'],
    ['BANDAR BARU KOTA SRI MAS', 'SEREMBAN', '71800'],['BANDAR NILAI UTAMA', 'SEREMBAN', '71800'],['BANDAR SRI SENDAYAN', 'SEREMBAN', '71950'],['PEKAN BROGA', 'SEREMBAN', '71750'],['PEKAN ULU BERANANG', 'SEREMBAN', '71760'],['PEKAN LABU', 'SEREMBAN', '71900'],
    ['PEKAN LENGGENG', 'SEREMBAN', '71770'],['PEKAN MAMBAU', 'SEREMBAN', '70300'],['PEKAN NILAI', 'SEREMBAN', '71800'],['PEKAN MANTIN', 'SEREMBAN', '71700'],['PEKAN PAJAM', 'SEREMBAN', '71700'],['PEKAN RANTAU', 'SEREMBAN', '71200'],['PEKAN TIROI', 'SEREMBAN', '71800'],
    ['PEKAN PANCOR', 'SEREMBAN', '70400'],['PEKAN TAMAN SEREMBAN', 'SEREMBAN', '70300'],['PEKAN RAHANG BARU', 'SEREMBAN', '70100'],['PEKAN PAROI', 'SEREMBAN', '70400'],['PEKAN BUKIT KEPAYANG', 'SEREMBAN', '70200'],['PEKAN DUSUN SETIA', 'SEREMBAN', '70400'],
    ['PEKAN SUNGAI GADUT', 'SEREMBAN', '71450'],['PEKAN BUKTI', 'SEREMBAN', '70300'],['PEKAN SIKAMAT', 'SEREMBAN', '70400'],['PEKAN SETUL', 'SEREMBAN', '71800'],['PEKAN SHAH BANDAR', 'SEREMBAN', '70300'],['PEKAN ULU TEMIANG', 'SEREMBAN', '70200'],
    ['PEKAN PAROI JAYA', 'SEREMBAN', '70400'],['PEKAN RASAH JAYA', 'SEREMBAN', '70300'],['PEKAN SENAWANG', 'SEREMBAN', '70450'],['PEKAN SEREMBAN JAYA', 'SEREMBAN', '70450'],['AYER KUNING', 'TAMPIN', '73200'],['GEMAS', 'TAMPIN', '73400'],['KERU', 'TAMPIN', '73000'],
    ['REPAH', 'TAMPIN', '73000'],['TEBONG', 'TAMPIN', '73000'],['BANDAR GEMAS', 'TAMPIN', '73400'],['BANDAR TAMPIN', 'TAMPIN', '73000'],['GEMENCHEH', 'TAMPIN', '73200'],['TAMPIN TENGAH', 'TAMPIN', '73000'],['PEKAN AIR KUNING SELATAN', 'TAMPIN', '73200'],
    ['PEKAN BATANG MELAKA', 'TAMPIN', '73100'],['PEKAN GEMENCHECH BAHRU', 'TAMPIN', '73200'],['PEKAN REPAH', 'TAMPIN', '73000'],['PEKAN TAMPIN TENGAH', 'TAMPIN', '73000'],['PEKAN AIR KUNING', 'TAMPIN', '73200'],['PEKAN PASIR BESAR', 'TAMPIN', '73200'],
    ['PEKAN REPAH PERMAI', 'TAMPIN', '73000'],['PEKAN REPAH JAYA', 'TAMPIN', '73000'],['JELAI', 'JEMPOL', '73500'],['ROMPIN', 'JEMPOL', '73430'],['BANDAR SERTING', 'JEMPOL', '72100'],['KUALA JEMPOL', 'JEMPOL', '73500'],['SERTING HILIR', 'JEMPOL', '72120'],
    ['SERTING ULU', 'JEMPOL', '72120'],['BANDAR BAHAU', 'JEMPOL', '72100'],['BANDAR SERI JEMPOL', 'JEMPOL', '72120'],['PEKAN BAHAU', 'JEMPOL', '72100'],['PEKAN BATU KIKIR', 'JEMPOL', '72200'],['PEKAN KUALA JELAI', 'JEMPOL', '73500'],['PEKAN ROMPIN', 'JEMPOL', '73430'],
    ['PEKAN LADANG GEDDES', 'JEMPOL', '72100'],['PEKAN MAHSAN', 'JEMPOL', '72100'],['PEKAN SERTING TENGAH', 'JEMPOL', '72100']
]


for mukim in n9_mukim_district_postcode_null:
    mukim_district_state['postcode']=np.where((mukim_district_state['mukim']==mukim[0])&(mukim_district_state['district']==mukim[1]),mukim[2],mukim_district_state['postcode'])


mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='NEGERI SEMBILAN')][[ 'district', 'state', 'mukim']]
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='NEGERI SEMBILAN')&(mukim_district_state['district']=='JEMPOL')][[ 'district', 'state', 'mukim']]

#mukim_district_state.isnull().sum()


,district,state,mukim


In [144]:
johor_mukim_district_postcode_null = \
[
    ['BAGAN', 'BATU PAHAT', '83000'],['CHAAH BAHRU', 'BATU PAHAT', '83400'],['KAMPONG BAHRU', 'BATU PAHAT', '83500'],['LINAU', 'BATU PAHAT', '83600'],['LUBOK', 'BATU PAHAT', '83700'],['MINYAK BEKU', 'BATU PAHAT', '83010'],['PESERAI', 'BATU PAHAT', '83020'],
    ['SIMPANG KANAN', 'BATU PAHAT', '83030'],['SIMPANG KIRI', 'BATU PAHAT', '83040'],['SRI GADING', 'BATU PAHAT', '83300'],['SRI MEDAN', 'BATU PAHAT', '83420'],['SUNGAI KLUANG', 'BATU PAHAT', '83100'],['SUNGAI PUNGGOR', 'BATU PAHAT', '83110'],
    ['TANJONG SEMBRONG', 'BATU PAHAT', '83510'],['BANDAR AYER HTAM', 'BATU PAHAT', '83610'],['BANDAR BANDAR PENGGARAM', 'BATU PAHAT', '83000'],['BANDAR RENGIT', 'BATU PAHAT', '83120'],['BANDAR SENGGARANG', 'BATU PAHAT', '83130'],['BANDAR YONG PENG', 'BATU PAHAT', '83700'],
    ['PLENTONG', 'JOHOR BAHRU', '81750'],['PULAI', 'JOHOR BAHRU', '81300'],['TEMBRAU', 'JOHOR BAHRU', '81100'],['BANDAR JOHOR BAHRU', 'JOHOR BAHRU', '80000'],['JELUTONG', 'JOHOR BAHRU', '80250'],      # Based on Jalan Jelutong area listings
    ['SUNGAI TIRAM', 'JOHOR BAHRU', '81800'],  # Near Ulu Tiram village
    ['TANJUNG KUPANG', 'JOHOR BAHRU', '81550'], # Gelang Patah zone
    ['ULU BENUT', 'KLUANG', '86200'],['KAHANG', 'KLUANG', '86000'],['KLUANG', 'KLUANG', '86000'],['LAYANG-LAYANG', 'KLUANG', '81850'],['MACHAP', 'KLUANG', '86200'],['NIYOR', 'KLUANG', '86000'],         # Based on Kampung Nyior listings
    ['PALOH', 'KLUANG', '86600'],['RENGAM', 'KLUANG', '86300'],['BANDAR KLUANG', 'KLUANG', '86000'],['BANDAR PALOH', 'KLUANG', '86600'],['BANDAR RENGAM', 'KLUANG', '86300'],['ULU SUNGAI JOHOR', 'KOTA TINGGI', '81000'],['ULU SUNGAI SELIDI BESAR', 'KOTA TINGGI', '81900'],
    ['JOHOR LAMA', 'KOTA TINGGI', '81900'],['KAMBAU', 'KOTA TINGGI', '81900'],['KOTA TINGGI', 'KOTA TINGGI', '81900'],['PANTAI TIMUR', 'KOTA TINGGI', '81930'],['PENGGERANG', 'KOTA TINGGI', '81600'],['SELIDI BESAR', 'KOTA TINGGI', '81900'],
    ['SELIDI KECHIL', 'KOTA TINGGI', '81930'],['TANJUNG SURAT', 'KOTA TINGGI', '81600'],['BANDAR KOTA TINGGI', 'KOTA TINGGI', '81900'],['JEMALUANG', 'MERSING', '86800'],['LENGGOR', 'MERSING', '86800'],['MERSING', 'MERSING', '86800'],
    ['PADANG ENDAU', 'MERSING', '86900'],['PENYABONG', 'MERSING', '86900'],['PULAU AUR', 'MERSING', '86800'],['PULAU BABI', 'MERSING', '86800'],           # Also known as Pulau Babi Besar
    ['PULAU PEMANGGIL', 'MERSING', '86800'],['PULAU SIBU', 'MERSING', '86800'],['PULAU TINGGI', 'MERSING', '86800'],['SEMBRONG', 'MERSING', '86800'],['TENGGAROH', 'MERSING', '86800'],['TENGLU', 'MERSING', '86800'],['TRIANG', 'MERSING', '86800'],
    ['BANDAR JEMALUANG', 'MERSING', '86800'],['BANDAR MERSING', 'MERSING', '86800'],['BANDAR MERSING KANAN', 'MERSING', '86800'],['BANDAR PADANG ENDAU', 'MERSING', '86900'],['AYER HITAM', 'MUAR', '84000'],['BANDAR', 'MUAR', '84000'],  # Generic, assumed central Muar
    ['BUKIT KEPONG', 'MUAR', '84030'],['JALAN BAKRI', 'MUAR', '84000'],['JORAK', 'MUAR', '84300'],['LENGA', 'MUAR', '84040'],['PARIT BAKAR', 'MUAR', '84000'],['PARIT JAWA', 'MUAR', '84150'],['SRI MERANTI', 'MUAR', '84000'],  # Based on Taman Meranti listings
    ['SUNGAI BALANG', 'MUAR', '83600'],['SUNGAI RAYA & KAMPUNG BUKIT PASIR', 'MUAR', '84300'],['SUNGAI TERAP', 'MUAR', '84300'],['BANDAR BANDAR MAHARANI', 'MUAR', '84000'],['BANDAR BUKIT KEPONG', 'MUAR', '84030'],['BANDAR PANCHOR', 'MUAR', '84500'],
    ['BANDAR PARIT JAWA', 'MUAR', '84150'],['PEKAN BUKIT PASIR', 'MUAR', '84300'],['AYER BALOI', 'PONTIAN', '82100'],['AIR MASIN', 'PONTIAN', '82300'],['API-API', 'PONTIAN', '82000'],['BENUT', 'PONTIAN', '82200'],['JERAM BATU', 'PONTIAN', '81500'],
    ['PENGKALAN RAJA', 'PONTIAN', '81500'],['PONTIAN', 'PONTIAN', '82000'],['RIMBA TERJUN', 'PONTIAN', '82000'],['SERKAT', 'PONTIAN', '82030'],['SUNGAI KARANG', 'PONTIAN', '82000'],['SUNGEI PINGGAN', 'PONTIAN', '82200'],['BANDAR BENUT', 'PONTIAN', '82200'],
    ['BANDAR PONTIAN KECHIL', 'PONTIAN', '82000'],['PEKAN PEKAN NENAS', 'PONTIAN', '81500'],['BEKOK', 'SEGAMAT', '86500'],['BULOH KASAP', 'SEGAMAT', '85000'],['CHAAH', 'SEGAMAT', '85400'],['GEMAS', 'SEGAMAT', '85100'],['GEMEREH', 'SEGAMAT', '85000'],['JABI', 'SEGAMAT', '85000'],
    ['JEMENTAH', 'SEGAMAT', '85200'],['LABIS', 'SEGAMAT', '85300'],['POGOH', 'SEGAMAT', '85000'],['SERMIN', 'SEGAMAT', '85000'],['SUNGAI SEGAMAT', 'SEGAMAT', '85000'],['BANDAR BATU ANAM', 'SEGAMAT', '85100'],['BANDAR BEKOK', 'SEGAMAT', '86500'],
    ['BANDAR BULOH KASAP', 'SEGAMAT', '85000'],['BANDAR JEMENTAH', 'SEGAMAT', '85200'],['BADNAR LABIS', 'SEGAMAT', '85300'],           # Typo noted: assuming "BANDAR LABIS"
    ['BANDAR SEGAMAT', 'SEGAMAT', '85000'],['PEKAN GEMAS BAHRU', 'SEGAMAT', '85100'],['KULAI', 'KULAIJAYA', '81000'],['SENAI', 'KULAIJAYA', '81400'],['SEDENAK', 'KULAIJAYA', '81000'],['BUKIT BATU', 'KULAIJAYA', '81000'],['BANDAR KULAI', 'KULAIJAYA', '81000'],
    ['KUNDANG', 'LEDANG', '84600'],['BADNAR BUKIT KANGKAR', 'LEDANG', '84400'],  # Corrected spelling from "BADNAR"
    ['PEKAN GRISEK', 'LEDANG', '84700'],['GERSIK', 'LEDANG', '84700'],['TANGKAK', 'LEDANG', '84900'],['BUKIT SERAMPANG', 'LEDANG', '85200'],       # Located near Jementah
    ['SEROM', 'LEDANG', '84400'],['KESANG', 'LEDANG', '84000'],                # Often overlaps with Muar
    ['BANDAR PARIT BUNGA', 'LEDANG', '84000'],['BANDAR SEROM', 'LEDANG', '84400'],['BANDAR SUNGAI MATI', 'LEDANG', '84400'],['BANDAR TANGKAK', 'LEDANG', '84900']
]


for mukim in johor_mukim_district_postcode_null:
    mukim_district_state['postcode']=np.where((mukim_district_state['mukim']==mukim[0])&(mukim_district_state['district']==mukim[1]),mukim[2],mukim_district_state['postcode'])

mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='JOHOR')][['district', 'state', 'mukim']]
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='JOHOR')&(mukim_district_state['district']=='LEDANG')][['district', 'state', 'mukim']]

#mukim_district_state.isnull().sum()

,district,state,mukim


In [145]:
trg_mukim_district_postcode_null = \
[
    ['BANDI', 'KUALA TERENGGANU', '24000'],['BANGGUL', 'KUALA TERENGGANU', '24000'],['HULU JABUR', 'KUALA TERENGGANU', '24060'],['PASIR SEMUT', 'KUALA TERENGGANU', '24000'],['TEBAK', 'KUALA TERENGGANU', '24000'],['TELUK KALUNG', 'KUALA TERENGGANU', '24100'],
    ['BATU BURUK', 'KUALA TERENGGANU', '20400'],['BATU RAKIT', 'KUALA TERENGGANU', '21020'],['CABANG TIGA', 'KUALA TERENGGANU', '21000'],['CENERIANG', 'KUALA TERENGGANU', '21080'],['GELUGUR KEDAI', 'KUALA TERENGGANU', '20050'],['GELUGUR RAJA', 'KUALA TERENGGANU', '20050'],
    ['KEPUNG', 'KUALA TERENGGANU', '20050'],['KUALA IBAI', 'KUALA TERENGGANU', '20400'],['KUALA NERUS', 'KUALA TERENGGANU', '21300'],['PALUH', 'KUALA TERENGGANU', '20050'],['PENGADANG BULUH', 'KUALA TERENGGANU', '21070'],['PULAU-PULAU', 'KUALA TERENGGANU', '20100'],
    ['RENGAS', 'KUALA TERENGGANU', '20050'],['SERADA', 'KUALA TERENGGANU', '20050'],['TOK JAMAL', 'KUALA TERENGGANU', '20050'],['BANDAR KUALA TERENGGANU', 'KUALA TERENGGANU', '20000'],['KUBANG PARIT', 'KUALA TERENGGANU', '20050'],['LOSONG', 'KUALA TERENGGANU', '21000'],
    ['PEKAN CABANG TIGA', 'KUALA TERENGGANU', '21000'],['ATAS TOL', 'KUALA TERENGGANU', '21070'],['BELARA', 'KUALA TERENGGANU', '21200'],['BUKIT BESAR', 'KUALA TERENGGANU', '21100'],['MANIR', 'KUALA TERENGGANU', '21200'],['PULAU REDANG', 'KUALA TERENGGANU', '21090'],
    ['HULU BERANG', 'HULU TERENGGANU', '21800'],['HULU TELEMUNG', 'HULU TERENGGANU', '21700'],['HULU TERENGGANU', 'HULU TERENGGANU', '21700'],['JENAGUR', 'HULU TERENGGANU', '21700'],['KUALA TELEMUNG', 'HULU TERENGGANU', '21210'],['PENGHULU DIMAN', 'HULU TERENGGANU', '21010'],
    ['TANGGUL', 'HULU TERENGGANU', '21040'],['TERSAT', 'HULU TERENGGANU', '21800'],['KUALA BERANG', 'HULU TERENGGANU', '21700'],['PEKAN KUALA BERANG', 'HULU TERENGGANU', '21700'],['BUKIT KENAK', 'BESUT', '22000'],['BUKIT PUTERI', 'BESUT', '22000'],['HULU BESUT', 'BESUT', '22200'],
    ['JABI', 'BESUT', '22020'],['KAMPONG RAJA', 'BESUT', '22200'],['KELUANG', 'BESUT', '22200'],['KERANDANG', 'BESUT', '22300'],['KUALA BESUT', 'BESUT', '22300'],['KUBANG BEMBAN', 'BESUT', '22000'],['LUBUK KAWAH', 'BESUT', '22000'],['PASIR AKAR', 'BESUT', '22000'],
    ['PELAGAT', 'BESUT', '22000'],['PENGKALAN NANGKA', 'BESUT', '22300'],['PULAU PERHENTIAN', 'BESUT', '22300'],['TEMBILA', 'BESUT', '22200'],['TENANG', 'BESUT', '22020'],['PEKAN JERTIH', 'BESUT', '22000'],['PEKAN KAMPUNG RAJA', 'BESUT', '22200'],
    ['PEKAN KUALA BESUT', 'BESUT', '22300'],['KUALA ABANG', 'DUNGUN', '23050'],['BESUL', 'DUNGUN', '23000'],['HULU PAKA', 'DUNGUN', '23100'],['JENGAI', 'DUNGUN', '23000'],['JERANGAU', 'DUNGUN', '23200'],['KUALA DUNGUN', 'DUNGUN', '23000'],['KUALA PAKA', 'DUNGUN', '23100'],
    ['KUMPAL', 'DUNGUN', '23000'],['PASIR RAJA', 'DUNGUN', '23000'],['RASAU', 'DUNGUN', '23200'],['SURA', 'DUNGUN', '23000'],['BANDAR DUNGUN', 'DUNGUN', '23000'],['PEKAN KUALA PAKA', 'DUNGUN', '23100'],['JERUNG', 'MARANG', '21600'],['MERCANG', 'MARANG', '21610'],
    ['PULAU KERENGGA', 'MARANG', '21600'],['RUSILA', 'MARANG', '21080'],['ALUR LIMBAT', 'MARANG', '21400'],['BUKIT PAYUNG', 'MARANG', '21400'],['PEKAN MARANG', 'MARANG', '21600'],['PEKAN BUKIT PAYUNG', 'MARANG', '21400'],['CALUK', 'SETIU', '21450'],       # Also spelled "Chalok"
    ['GUNTUNG', 'SETIU', '22100'],     # Kampung Guntung Dalam is listed under 22100
    ['HULU NERUS', 'SETIU', '22100'],  # Inferred from nearby Kampung Alor Serdang
    ['MERANG', 'SETIU', '21010'],      # Confirmed via SK Merang listings
    ['TASIK', 'SETIU', '22120'],        # Based on Permaisuri coverage
    ['BANDI', 'KEMAMAN', '24200'],['BANGGUL', 'KEMAMAN', '24000'],['HULU JABUR', 'KEMAMAN', '24060'],['PASIR SEMUT', 'KEMAMAN', '24000'],['TEBAK', 'KEMAMAN', '24000'],['TELUK KALUNG', 'KEMAMAN', '24000'],['KEMASIK', 'KEMAMAN', '24200'],['PEKAN AIR JERNIH', 'KEMAMAN', '24200'],
    ['PEKAN AIR PUTIH', 'KEMAMAN', '24050'],['PEKAN KEMASIK', 'KEMAMAN', '24200'],['BINJAI', 'KEMAMAN', '24000'],['CUKAI', 'KEMAMAN', '24000'],['HULU CUKAI', 'KEMAMAN', '24000'],['KERTEH', 'KEMAMAN', '24300'],['KIJAL', 'KEMAMAN', '24100'],['BANDAR CUKAI', 'KEMAMAN', '24000'],
    ['PEKAN KIJAL', 'KEMAMAN', '24100']
]


for mukim in trg_mukim_district_postcode_null:
    mukim_district_state['postcode']=np.where((mukim_district_state['mukim']==mukim[0])&(mukim_district_state['district']==mukim[1]),mukim[2],mukim_district_state['postcode'])


mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='TERENGGANU')][['district', 'state', 'mukim']]
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='TERENGGANU')&(mukim_district_state['district']=='KEMAMAN')][['district', 'state', 'mukim']]

#mukim_district_state.isnull().sum()

,district,state,mukim


In [147]:
phg_mukim_district_postcode_null = \
[
    ['BENTONG', 'BENTONG', '28700'],['SABAI', 'BENTONG', '28620'],           # Ladang Sabai is listed under Karak with postcode 28620
    ['PELANGAI', 'BENTONG', '28750'],        # Pelangai has its own postcode distinct from Bentong town
    ['BANDAR BENTONG', 'BENTONG', '28700'],['BANDAR KARAK', 'BENTONG', '28600'],['PEKAN TELEMONG', 'BENTONG', '28500'],['HULU TELOM', 'CAMERON HIGHLANDS', '39000'],['RINGLET', 'CAMERON HIGHLANDS', '39200'],['TANAH RATA', 'CAMERON HIGHLANDS', '39000'],
    ['BANDAR TANAH RATA', 'CAMERON HIGHLANDS', '39000'],['PEKAN BERINCHANG', 'CAMERON HIGHLANDS', '39100'],['PEKAN LUBOK TAMANG', 'CAMERON HIGHLANDS', '39200'],['PEKAN RINGLET', 'CAMERON HIGHLANDS', '39200'],['BURAU', 'JERANTUT', '27050'],['ULU CHEKA', 'JERANTUT', '27010'],
    ['ULU TEMBELING', 'JERANTUT', '27000'],['KELOLA', 'JERANTUT', '27090'],['KUALA TEMBELING', 'JERANTUT', '27020'],['PEDAH', 'JERANTUT', '27150'],['PULAU TAWAR', 'JERANTUT', '27050'],['TEBING TINGGI', 'JERANTUT', '27000'],['TEH', 'JERANTUT', '27000'],
    ['TEMBELING', 'JERANTUT', '27000'],['BANDAR JERANTUT', 'JERANTUT', '27000'],['PEKAN KUALA TEMBELING', 'JERANTUT', '27020'],['PEKAN JERANSANG', 'JERANTUT', '27000'],['ULU LEPAR', 'KUANTAN', '26330'],['ULU KUANTAN', 'KUANTAN', '26200'],['KUALA KUANTAN', 'KUANTAN', '25100'],
    ['PENOR', 'KUANTAN', '26060'],['BANDAR KUANTAN', 'KUANTAN', '25000'],['BESERAH', 'KUANTAN', '26100'],['SUNGAI KARANG', 'KUANTAN', '26100'],['BANDAR GAMBANG', 'KUANTAN', '26100'],               # ✅ corrected from 26300
    ['PEKAN BESERAH', 'KUANTAN', '26100'],['PEKAN TANJNG LUMPUR', 'KUANTAN', '26060'],['BATU YON', 'LIPIS', '27200'],['BUDU', 'LIPIS', '27310'],['CHEKA', 'LIPIS', '27200'],['GUA', 'LIPIS', '27210'],['ULU JELAI', 'LIPIS', '27200'],['KECHAU', 'LIPIS', '27200'],
    ['TELANG', 'LIPIS', '27200'],['KUALA LIPIS', 'LIPIS', '27200'],['PENJOM', 'LIPIS', '27200'],['TANJUNG BESAR', 'LIPIS', '27300'],['BANDAR KUALA LIPIS', 'LIPIS', '27200'],['BENTA', 'LIPIS', '27300'],['PADANG TENGKU', 'LIPIS', '27100'],
    ['PEKAN PADANG TENGKU', 'LIPIS', '27100'],['PEKAN TAMAN JELAI', 'LIPIS', '27200'],['PEKAN PENJOM', 'LIPIS', '27200'],['PEKAN MELA', 'LIPIS', '27010'],['PEKAN KERAMBIT', 'LIPIS', '27200'],['PEKAN RPSB KG. PAGAR', 'LIPIS', '27200'],
    ['PEKAN MERAPUH', 'LIPIS', '27210'],['PEKAN KECHAU TUI', 'LIPIS', '27200'],['BEBAR', 'PEKAN', '26600'],['GANCHONG', 'PEKAN', '26630'],           # ✅ corrected from 26600
    ['KUALA PAHANG', 'PEKAN', '26660'],       # ✅ corrected from 26600
    ['LEPAR', 'PEKAN', '26650'],              # ✅ corrected from 26600
    ['PAHANG TUA', 'PEKAN', '26620'],         # ✅ corrected from 26600
    ['PEKAN', 'PEKAN', '26600'],['PENYOR', 'PEKAN', '26600'],['PULAU MANIS', 'PEKAN', '26640'],        # ✅ corrected from 26600
    ['PULAU RUSA', 'PEKAN', '26630'],         # ✅ corrected from 26600
    ['TEMAI', 'PEKAN', '26640'],              # ✅ corrected from 26600
    ['BANDAR PEKAN', 'PEKAN', '26600'],['PEKAN KUALA PAHANG', 'PEKAN', '26660'], # ✅ corrected from 26600
    ['PEKAN NENASI', 'PEKAN', '26680'],        # ✅ corrected from 26700
    ['BATU TALAM', 'RAUB', '27610'],['DONG', 'RAUB', '27400'],['GALI', 'RAUB', '27600'],['HULU DONG', 'RAUB', '27400'],['SEGA', 'RAUB', '27660'],['SEMANTAN ULU', 'RAUB', '27600'],['TERAS', 'RAUB', '27670'],['BANDAR RAUB', 'RAUB', '27600'],['BANDAR TERAS', 'RAUB', '27670'],
    ['PEKAN CHEROH', 'RAUB', '27620'],['PEKAN DONG', 'RAUB', '27400'],['PEKAN TRANUM', 'RAUB', '27670'],['BANGAU', 'TEMERLOH', '28000'],['JENDERAK', 'TEMERLOH', '28050'],['KERDAU', 'TEMERLOH', '28020'],['LEBAK', 'TEMERLOH', '28000'],['LIPAT KIJANG', 'TEMERLOH', '28010'],
    ['MENTAKAB', 'TEMERLOH', '28400'],['PERAK', 'TEMERLOH', '28000'],  # Likely refers to Kampung Perak in Temerloh
    ['SANGGANG', 'TEMERLOH', '28020'],['SEMANTAN', 'TEMERLOH', '28000'],['SONGSANG', 'TEMERLOH', '28000'],['BANDAR MENTAKAB', 'TEMERLOH', '28400'],['BANDAR TEMERLOH', 'TEMERLOH', '28000'],['PEKAN KERDAU', 'TEMERLOH', '28020'],['PEKAN KUALA KERAU', 'TEMERLOH', '28000'],
    ['LANCHANG', 'TEMERLOH', '28500'],['KERATONG', 'ROMPIN', '26900'],['ROMPIN', 'ROMPIN', '26800'],['TIOMAN', 'ROMPIN', '26800'],['BANDAR ROMPIN I', 'ROMPIN', '26800'],['BANDAR ROMPIN II', 'ROMPIN', '26800'],['BANDAR ROMPIN III', 'ROMPIN', '26800'],
    ['BANDAR ROMPIN IV', 'ROMPIN', '26800'],['BANDAR TIOMAN', 'ROMPIN', '26800'],['ENDAU', 'ROMPIN', '26800'],['PONTIAN', 'ROMPIN', '26800'],['BANDAR PONTIAN', 'ROMPIN', '26800'],['BANDAR ENDAU', 'ROMPIN', '26820'],['BANDAR MUADZAM SHAH I', 'ROMPIN', '26700'],
    ['BANDAR MUADZAM SHAH II', 'ROMPIN', '26700'],['PEKAN KUALA ROMPIN', 'ROMPIN', '26800'],['PEKAN TIOMAN', 'ROMPIN', '26800'],['BUKIT SEGUMPAL', 'MARAN', '28000'],['CHENOR', 'MARAN', '28100'],['KERTAU', 'MARAN', '28100'],['LUIT', 'MARAN', '26500'],
    ['BANDAR MARAN', 'MARAN', '26500'],['PEKAN CHENOR', 'MARAN', '28100'],['PEKAN SRI JAYA', 'MARAN', '26500'],['BERA', 'BERA', '28200'],['TRIANG', 'BERA', '28300'],['BANDAR TRIANG', 'BERA', '28300'],['PEKAN DURIAN TAWAR', 'BERA', '28200'],['PEKAN MENGKARAK', 'BERA', '28200'],
    ['PEKAN MENGKUANG', 'BERA', '28200']
]

for mukim in phg_mukim_district_postcode_null:
    mukim_district_state['postcode']=np.where((mukim_district_state['mukim']==mukim[0])&(mukim_district_state['district']==mukim[1]),mukim[2],mukim_district_state['postcode'])


mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PAHANG')][['district', 'state', 'mukim']]
#mukim_district_state[(mukim_district_state['postcode'].isnull())&(mukim_district_state['state']=='PAHANG')&(mukim_district_state['district']=='BENTONG')][['district', 'state', 'mukim']]
#mukim_district_state.isnull().sum()


,district,state,mukim


In [148]:
mukim_district_state[mukim_district_state['postcode'].isnull()]

,postcode,location,district,state,mukim
1422,NaN,NaN,W. P. PUTRAJAYA,WP PUTRAJAYA,BANDAR PUTRAJAYA
1881,NaN,NaN,KOTA KINABALU,SABAH,KEPAYAN
1885,NaN,NaN,KOTA KINABALU,SABAH,SEPANGGAR
1908,NaN,NaN,NaN,SABAH,DONGGONGON


In [150]:
mukim_district_state['district'] = np.where(mukim_district_state['mukim']=='DONGGONGON', 'PENAMPANG', mukim_district_state['district'])

In [151]:
etc_mukim_district_postcode_null = \
[
    ['BANDAR PUTRAJAYA', 'W. P. PUTRAJAYA', '62000'],     # Central Putrajaya zone
    ['KEPAYAN', 'KOTA KINABALU', '88200'],                # Kampung Kepayan confirmed under 88200
    ['SEPANGGAR', 'KOTA KINABALU', '88450'],              # Taman Sepanggar and surrounding areas
    ['DONGGONGON', 'PENAMPANG', '89500']                  # Pekan Donggongon and Penampang core
]

for mukim in etc_mukim_district_postcode_null:
    mukim_district_state['postcode']=np.where((mukim_district_state['mukim']==mukim[0])&(mukim_district_state['district']==mukim[1]),mukim[2],mukim_district_state['postcode'])


mukim_district_state[(mukim_district_state['postcode'].isnull())][['district', 'state', 'mukim']]

,district,state,mukim


- Check if No Null Value for Postcode

In [152]:
mukim_district_state[(mukim_district_state['postcode'].isnull())]

,postcode,location,district,state,mukim


In [153]:
mukim_district_state.head(10)

,postcode,location,district,state,mukim
0,01000,NaN,KANGAR,PERLIS,ABI
1,02600,NaN,ARAU,PERLIS,ARAU
2,02400,NaN,KANGAR,PERLIS,BERSERI
3,02500,NaN,PADANG BESAR,PERLIS,CHUPING
4,01000,NaN,KANGAR,PERLIS,UTAN AJI
5,01000,NaN,ARAU,PERLIS,JEJAWI
6,02000,NaN,KANGAR,PERLIS,KAYANG
7,01000,NaN,KANGAR,PERLIS,KECHOR
8,02000,NaN,KANGAR,PERLIS,KUALA PERLIS
9,02600,NaN,ARAU,PERLIS,KURONG ANAI


In [154]:
mukim_district_state[mukim_district_state['postcode'].str.len()>5]

,postcode,location,district,state,mukim
299,16070,NaN,BACHOK,KELANTAN,PEKAN JELAWAT
416,17010,NaN,PASIR MAS,KELANTAN,ALOR BULOH
544,16210,NaN,TUMPAT,KELANTAN,WAKAF BHARU
1901,89260,NaN,TUARAN,SABAH,TENGHILAN
1905,89200,NaN,TUARAN,SABAH,TAMPARULI
1907,89800,NaN,BEAUFORT,SABAH,MEMBAKUT


In [155]:
mukim_district_state['postcode'] = mukim_district_state['postcode'].str.strip()
mukim_district_state[mukim_district_state['postcode'].str.len()>5]

,postcode,location,district,state,mukim


In [170]:
location.head(10)

,postcode,location,district,state
0,01000,ALOR REDEH,KANGAR,PERLIS
1,01000,ALOR TOK TERANG,KANGAR,PERLIS
2,01000,BOHOR GELAM,KANGAR,PERLIS
3,01000,JALAN ABI TOK HASHIM,KANGAR,PERLIS
4,01000,JALAN ALOR SENA,KANGAR,PERLIS
5,01000,JALAN ARAU,KANGAR,PERLIS
6,01000,JALAN BESAR KANGAR,KANGAR,PERLIS
7,01000,JALAN BOHOR GUNCAR,KANGAR,PERLIS
8,01000,JALAN BUKIT MERANTI,KANGAR,PERLIS
9,01000,JALAN HOME GUARD,KANGAR,PERLIS


In [171]:
mukim_district_state.head(10)

,postcode,location,district,state,mukim
0,01000,NaN,KANGAR,PERLIS,ABI
1,02600,NaN,ARAU,PERLIS,ARAU
2,02400,NaN,KANGAR,PERLIS,BERSERI
3,02500,NaN,PADANG BESAR,PERLIS,CHUPING
4,01000,NaN,KANGAR,PERLIS,UTAN AJI
5,01000,NaN,ARAU,PERLIS,JEJAWI
6,02000,NaN,KANGAR,PERLIS,KAYANG
7,01000,NaN,KANGAR,PERLIS,KECHOR
8,02000,NaN,KANGAR,PERLIS,KUALA PERLIS
9,02600,NaN,ARAU,PERLIS,KURONG ANAI


In [175]:
location_mukim_district_state=pd.concat([mukim_district_state, location])
location_mukim_district_state.head(10)

,postcode,location,district,state,mukim
0,01000,NaN,KANGAR,PERLIS,ABI
1,02600,NaN,ARAU,PERLIS,ARAU
2,02400,NaN,KANGAR,PERLIS,BERSERI
3,02500,NaN,PADANG BESAR,PERLIS,CHUPING
4,01000,NaN,KANGAR,PERLIS,UTAN AJI
5,01000,NaN,ARAU,PERLIS,JEJAWI
6,02000,NaN,KANGAR,PERLIS,KAYANG
7,01000,NaN,KANGAR,PERLIS,KECHOR
8,02000,NaN,KANGAR,PERLIS,KUALA PERLIS
9,02600,NaN,ARAU,PERLIS,KURONG ANAI


In [176]:
len(location_mukim_district_state)

58394

In [177]:
location_mukim_district_state.to_csv('../output/malaysia-postcodes-location-mukim-district-state.csv', index=False)